In [3]:
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import math
import numpy as np
from torchvision import datasets, transforms
import torch
from numpy import *

from utils.sampling import mnist_iid_cluster, mnist_noniid_cluster, cifar_iid_cluster,cifar_noniid_cluster
from models_v4.Update import LocalTrain,ClusterDetect
from models_v4.Nets import CNNMnist, CNNCifar, CNNMnist2
from models_v4.Fed import FedAvg_vectorization
from models_v4.test import test_acc
from scipy.linalg import null_space


%load_ext autoreload
%autoreload 2
class my_argument:    
    epochs = 10   #"rounds of training"
    num_users = 5 # "number of users: K"
    frac = 0.1 #"the fraction of clients: C"
    local_ep=5 #"the number of local epochs: E"
    local_bs=50 #"local batch size: B"
    bs=128 #"test batch size"
    lr=0.0001 #"learning rate"
    momentum=0.5 # "SGD momentum (default: 0.5)"
    split='user' # "train-test split type, user or sample"

    # model arguments
    model = 'cnn'
    
    # other arguments
    dataset='cifar' #, help="name of dataset")
    iid=0
    num_classes=10#, help="number of classes")
    num_channels=1#, help="number of channels of images")
    gpu=1#, help="GPU ID, -1 for CPU")
    stopping_rounds=10#, help='rounds of early stopping')
    verbose='False'#, help='verbose print')
    seed=1#, help='random seed (default: 1)')
    cluster=5
    opt='ADAM'
args = my_argument()
#args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')
print(args.epochs)
np.random.seed(1)


10


In [4]:
encoder1=[]
encoder2=[]
decoder1=[]
decoder2=[]
A_bar1=[]
A_bar2=[]
U1=[]
U2=[]
mu=0
sigma=1
#alpha=0.001
for i in range(args.cluster):
    encoder1.append([])
    decoder1.append([])
    encoder2.append([])
    decoder2.append([])
    A_bar1.append([])
    A_bar2.append([])
    U1.append([])
    U2.append([])
    decoder1.append([])
    decoder2.append([])
part=1000; # the size of vector that we want to send each time
total_size=round(62006*1) #total length of the vector
d1=part
slot=math.ceil(total_size/part) # the no. of slots we need to send vectors
d2=total_size%part
slot1=math.ceil(total_size/d1)-1
slot2=total_size%d1
print(slot)
print(d2)
K=args.cluster
for i in range(args.cluster):
    num1=np.random.normal(mu, sigma, K*d1*d1)
    #num1=np.random.rand(K*d1*d1)
    num2=np.random.normal(mu, sigma, K*d2*d2)
    #num2=np.random.rand(K*d2*d2)
    J1=np.random.rand(d1,d1)
    for j in range(K):
        if (j!=i):
            encoder=np.zeros((d1,d1))
            J1=np.vstack([J1,encoder])
        else:
            encoder=np.identity(d1)
            J1=np.vstack([J1,encoder])
    J1 = np.delete(J1,np.s_[0:d1], axis=0)
    encoder1[i]=J1
    #encoder2[i]=num2.reshape(K*d2,d2)
    #divide the vector into 15 parts of size 10000 each and one part of size 9010
#define encoder[0],encoder[1],encoder[2],encoder[3]
K=args.cluster
print(encoder1[1].shape)
#N_R=5#no of receiver antenna
for i in range(K): # K= no. of clusters
    J1=np.random.rand(K*d1,d1)
    #J2=np.random.rand(K*d2,d2)
    #print(type(J2))
    print(type(encoder1[i]))
    for j in range(args.cluster):
        if(j!=i):
            J1=np.hstack([J1,encoder1[j]]) #concatenating different encoder matrices
            #J1.append(encoder1[j])
            #J2=np.hstack([J2,encoder2[j]])
    print(J1.shape)
    J1 = np.delete(J1,np.s_[0:d1], axis=1)# delete the initial d rows in J
    print(J1.shape)
    #J2 = np.delete(J2,np.s_[0:d2], axis=1)
    #print(J2.shape)
    A_bar1[i]=J1
    #A_bar2[i]=J2
    print(A_bar1[i].shape)
    #print(A_bar2[i].shape)
    print("printing type of A_bar")
    print(type(A_bar1[i]))
    U1[i]=null_space((A_bar1[i].transpose()))
    print(U1[i].shape)
    #U2[i]=null_space((A_bar2[i].transpose()))
    #print(U2[i].shape)
    decoder1[i]=(np.linalg.inv((U1[i].transpose())@(encoder1[i])) @(U1[i].transpose()))
    #decoder2[i]=(np.linalg.inv((U2[i].transpose())@(encoder2[i])) @(U2[i].transpose()))
    #decoder1[i]=
    print("printing size of decoder1")
    print(decoder1[0].shape)
    print(type(decoder1))
    #print("printing size of decoder2")
    #print(decoder2[0].shape)


63
6
(5000, 1000)
<class 'numpy.ndarray'>
(5000, 5000)
(5000, 4000)
(5000, 4000)
printing type of A_bar
<class 'numpy.ndarray'>
(5000, 1000)
printing size of decoder1
(1000, 5000)
<class 'list'>
<class 'numpy.ndarray'>
(5000, 5000)
(5000, 4000)
(5000, 4000)
printing type of A_bar
<class 'numpy.ndarray'>
(5000, 1000)
printing size of decoder1
(1000, 5000)
<class 'list'>
<class 'numpy.ndarray'>
(5000, 5000)
(5000, 4000)
(5000, 4000)
printing type of A_bar
<class 'numpy.ndarray'>
(5000, 1000)
printing size of decoder1
(1000, 5000)
<class 'list'>
<class 'numpy.ndarray'>
(5000, 5000)
(5000, 4000)
(5000, 4000)
printing type of A_bar
<class 'numpy.ndarray'>
(5000, 1000)
printing size of decoder1
(1000, 5000)
<class 'list'>
<class 'numpy.ndarray'>
(5000, 5000)
(5000, 4000)
(5000, 4000)
printing type of A_bar
<class 'numpy.ndarray'>
(5000, 1000)
printing size of decoder1
(1000, 5000)
<class 'list'>


In [5]:
# load dataset and split users
if args.dataset == 'mnist':
    trans_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    dataset_train = datasets.MNIST('./data/mnist/', train=True, download=True, transform=trans_mnist)
    dataset_test = datasets.MNIST('./data/mnist/', train=False, download=True, transform=trans_mnist)
    count=0
    #print(len(dataset_train))
    dict_users=[] #2D array in each row, users of a particular cluster
    train_data=[]
    test_data=[]
    for j in range(args.cluster):
        train_data.append([])
        test_data.append([])
        dict_users.append([])
    for j in range(len(dataset_train)):
        data,label=j
        if (label==0) | (label==1):
            train_data[0].append(dataset_train[j])
        elif (label==2) | (label==3):
            train_data[1].append(dataset_train[j])
        elif (label==4) | (label==5):
            train_data[2].append(dataset_train[j])
        elif (label==6) | (label==7):
            train_data[3].append(dataset_train[j])
        elif (label==8) | (label==9):
            train_data[4].append(dataset_train[j])
    for j in range(len(dataset_test)):
        data,label=j
        if (label==0) | (label==1):
            test_data[0].append(dataset_test[j])
        elif (label==2) | (label==3):
            test_data[1].append(dataset_test[j])
        elif (label==4) | (label==5):
            test_data[2].append(dataset_test[j])
        elif (label==6) | (label==7):
            test_data[3].append(dataset_test[j])
        elif (label==8) | (label==9):
            test_data[4].append(dataset_test[j])
    
#defining 5 different types of datasets for 5 different clusters
    
    if args.iid:
        for cluster_no in range(args.cluster):
            dict_users[cluster_no] = mnist_iid_cluster(train_data[cluster_no], args.num_users)
    else:
        for cluster_no in range(args.cluster):
            dict_users[cluster_no] = mnist_noniid_cluster(train_data[cluster_no], args.num_users)
elif args.dataset == 'cifar':
    trans_cifar = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    dataset_train = datasets.CIFAR10('./data/cifar', train=True, download=True, transform=trans_cifar)
    dataset_test = datasets.CIFAR10('./data/cifar', train=False, download=True, transform=trans_cifar)
#defining 5 different types of datasets for 5 different clusters
    count=0
    #print(len(dataset_train))
    dict_users=[] #2D array in each row, users of a particular cluster
    train_data=[]
    test_data=[]
    for j in range(args.cluster):
        train_data.append([])
        test_data.append([])
        dict_users.append([])
    for j in range(len(dataset_train)):
        data,label=dataset_train[j]
        if (label==0) | (label==1):
            train_data[0].append(dataset_train[j])
        elif (label==2) | (label==3):
            train_data[1].append(dataset_train[j])
        elif (label==4) | (label==5):
            train_data[2].append(dataset_train[j])
        elif (label==6) | (label==7):
            train_data[3].append(dataset_train[j])
        elif (label==8) | (label==9):
            train_data[4].append(dataset_train[j])
    for j in range(len(dataset_test)):
        data,label=dataset_test[j]
        if (label==0) | (label==1):
            test_data[0].append(dataset_test[j])
        elif (label==2) | (label==3):
            test_data[1].append(dataset_test[j])
        elif (label==4) | (label==5):
            test_data[2].append(dataset_test[j])
        elif (label==6) | (label==7):
            test_data[3].append(dataset_test[j])
        elif (label==8) | (label==9):
            test_data[4].append(dataset_test[j])

    if args.iid:
        for cluster_no in range(args.cluster):
            dict_users[cluster_no] = cifar_iid_cluster(train_data[cluster_no], args.num_users)
    else:
        for cluster_no in range(args.cluster):
            dict_users[cluster_no] = cifar_noniid_cluster(train_data[cluster_no], args.num_users)
else:
    exit('Error: dataset not found')
img_size = dataset_train[0][0].shape
#print(dict_users[0])
#print((dict_users[0][4]))
print(len(dataset_train))
print(len(dict_users))
print(len(train_data[0]))
#print(train_data[0])
#idxs_users = np.random.choice(range(args.num_users), m, replace=False)
#print(idxs_users)

Files already downloaded and verified
Files already downloaded and verified
50000
5
10000


In [7]:
use_cuda = torch.cuda.is_available()
#print(use_cuda)
args.device = torch.device("cuda:3" if use_cuda else "cpu")
#args.device=torch.device("cpu")
print(args.device)

cuda:3


In [8]:
acc_test=[]
acc_test_arr=[]
loss_test=[]
loss_test_arr=[]
for cluster_no in range(args.cluster):
    acc_test.append([])
    loss_test.append([])
    acc_test_arr.append([])
    loss_test_arr.append([])
for cluster_no in range(args.cluster):
    for i in range(args.cluster):
        acc_test_arr[cluster_no].append([])
        loss_test_arr[cluster_no].append([])
print(acc_test_arr)
noise_acc=[]
for user in range(25):
    noise_acc.append([])

[[[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []]]


In [10]:
# build model
from models_v4.Fed import weight_vectorization,FedSubstract,FedAvg_gradient,FedAdd
from models_v4.Fed import FedAdd,FedSubstract,weight_vectorization_gen,FedAvg_gradient
import numpy as np
import copy
if args.model == 'cnn' and args.dataset == 'cifar':
    net_glob=[]
    for i in range(args.cluster):
        net_glob.append(CNNCifar(args=args).to(args.device))
elif args.model == 'cnn' and args.dataset == 'mnist':
    net_glob=[]
    for i in range(args.cluster):
        net_glob.append(CNNMnist2(args=args).to(args.device))

else:
    exit('Error: model not found')
#print(net_glob)
w_glob=[]
# copy weights
for i in range(args.cluster):
    net_glob[i].train()
    w_glob.append(net_glob[i].state_dict())
abs_vect,layer_size=weight_vectorization_gen(w_glob[2])
net_glob_in=copy.deepcopy(net_glob)
w_glob_in=copy.deepcopy(w_glob)
print(len(abs_vect))
#print("weights")
#print(w_glob)
#print(w_glob.shape)
#print(w_glob[2])

62006


In [11]:
w_glob_in[0]=torch.load("net1_2.pt")
w_glob_in[1]=torch.load("net2_2.pt")
w_glob_in[2]=torch.load("net3_2.pt")
w_glob_in[3]=torch.load("net4_2.pt")
w_glob_in[4]=torch.load("net5_2.pt")
net_glob_in[0].load_state_dict(w_glob_in[0])
net_glob_in[1].load_state_dict(w_glob_in[1])
net_glob_in[2].load_state_dict(w_glob_in[2])
net_glob_in[3].load_state_dict(w_glob_in[3])
net_glob_in[4].load_state_dict(w_glob_in[4])

<All keys matched successfully>

In [13]:
# build model

# torch.save(net_glob_in[0].state_dict(),  "Model_cluster0.pt")
# torch.save(net_glob_in[1].state_dict(),  "Model_cluster1.pt")
# torch.save(net_glob_in[2].state_dict(),  "Model_cluster2.pt")
# torch.save(net_glob_in[3].state_dict(),  "Model_cluster3.pt")
# torch.save(net_glob_in[4].state_dict(),  "Model_cluster4.pt")
d=62006
args.lr=0.0002
from models_v4.Fed import weight_vectorization_cifar,FedAdd,FedSubstract,FedAvg_gradient
import numpy as np
import copy
#slot=math.ceil(d/d1) # the no. of slots we need to send vectors

# training
cv_loss, cv_acc = [], []
val_loss_pre, counter = 0, 0
net_best = None
best_loss = None
val_acc_list, net_list = [], []
num=[]
import random
part=1000
b=part
D=part #dimension of noise vector = Nr= Kd
mu=0
sigma=1
K=args.cluster
K_global=round(1*d)
error=[]
w_glob=copy.deepcopy(w_glob_in)
net_glob=copy.deepcopy(net_glob_in)
#hist_ = np.zeros(10,dtype=int)
sample=0 # fro the purpose of using fresh samples in each iteration
n=K*b*K*b
H = np.random.normal(loc=0, scale=np.sqrt(2)/2, size=(n, 2)).view(np.complex128)
H=real(H.reshape(K*b,K*b))
for iter in range(1000): #args.epochs
    print("iteration number",iter)
    if(iter%4==0):
        sample=0
    encoded=[]
    decoded=[]
    num=[]
    noise=[] #noise vector
  
    num=np.random.normal(mu, sigma, K*D)
    num=np.transpose(num.reshape(1,len(num)))
    #m = 10
    loss_train=[]
    cluster_block=[]
    noise_dec=[] #decoded noise vector
    recovered=[]
    for cluster_no in range(args.cluster):
        loss_train.append([])
        cluster_block.append([])
        decoded.append([])
        recovered.append([])
        noise_dec.append([])
    Power=[]
    Power2=[]
    alpha=[]
    feature_vector2=[]
    for i in range(args.cluster):
        Power.append([])
        Power2.append([])
        alpha.append([])
        feature_vector2.append(np.zeros((d,1)))
        
    #m_global=np.zeros((d,1))
    #location_global=np.random.choice(range(d),K_global,replace=False)
    #for i in location_global:
        #m_global[i]=1
                
    #idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    idx_users=[]
    sorted_train_data=[]
    sorted_test_data=[]
    for cluster_no in range(args.cluster):
        for index in dict_users[cluster_no]:
            idx_users.append(index) # putting the data indices of users in this list
            sorted_train_data.append(train_data[cluster_no])#putting the corresponding training data in this array
            sorted_test_data.append(test_data[cluster_no])
    for user in range(len(idx_users)): # no. of loop= no. of users
        cluster_loss=[]
        for i in range(args.cluster):
             # each type of dataset belong to 5 users
            local2 = ClusterDetect(args=args, dataset= sorted_train_data[user], idxs=idx_users[user][(sample*250):(sample+1)*250])
            #local2 = ClusterDetect(args=args, dataset= sorted_train_data[user], idxs=idx_users[user][0:599])
            # using 1st 600 data
            w2, loss2 = local2.train2(net=copy.deepcopy(net_glob[i]).to(args.device))
            cluster_loss.append(loss2)
        #print("printing user")
        #print(user)
        #print(cluster_loss)
        minimum=min(cluster_loss)
        index_of_min=cluster_loss.index(minimum)
        cluster_block[index_of_min].append(user)
    print(cluster_block[0])
    print(cluster_block[1])
    print(cluster_block[2])
    print(cluster_block[3])
    print(cluster_block[4])
    #K_global=part
    #m_global=np.zeros((d,1))
#     location_global=np.random.choice(range(d),K_global,replace=False)
#     for i in location_global:
#         m_global[i]=1
    updated=[]
    model_diff=[]
    grad_vect=[]
    prev=[]
    error=[]
    grad_vect_send=[]
    store_grad=[]
    sparse_vector=[]
    for i in range(25):
        updated.append([])
        model_diff.append([])
        grad_vect.append([])
        prev.append([])
        #error.append(np.zeros((d,1)))
        grad_vect_send.append([])
        store_grad.append([])
        sparse_vector.append([])
    b=part  
    sigma_R=1/math.sqrt(b)
    mu_R=0
    #R=np.random.normal(mu_R, sigma_R, (b,total_size))
    superposition=0
    #location_global=np.random.choice(range(d),K_global,replace=False) # storing top- K_local locations for each user
    #location_global=np.sort(location_global)
#     for i in location_global:
#         m_global[i]=1
    cluster_gradient=[]
    for cluster_no in range(args.cluster):
        w_locals, loss_locals,grad_locals,diff_locals= [],[],[],[]
        cluster_gradient.append([])
        if(cluster_block[cluster_no]==[]):
            continue
        norm=[]
        for user2 in cluster_block[cluster_no]:
            trans=np.array([])
            #local = LocalUpdate(args=args, dataset=sorted_train_data[user], idxs=idx_users[user][(sample+1)*600:(sample+2)*600])
            #local = LocalUpdate(args=args, dataset=sorted_train_data[user], idxs=idx_users[user][600:1199])
            #print(user2)
            total_P=0
            updated[user2]=copy.deepcopy(w_glob[cluster_no])
            local = LocalTrain(args=args, dataset=sorted_train_data[user2], idxs=idx_users[user2][(sample+1)*250:(sample+2)*250])
            #using 2nd half data
            w, loss = local.train(net=copy.deepcopy(net_glob[cluster_no]).to(args.device))
            w_locals.append(copy.deepcopy(w))
            loss_locals.append(copy.deepcopy(loss))
            prev[user2]=updated[user2]
            model_diff[user2]=FedSubstract(w,prev[user2])
            grad_vect[user2],layer_size=weight_vectorization_gen(model_diff[user2])
            #grad_vect[user2]=grad_vect[user2]+error[user2]
            #for i in location_global:
                #sparse_vector[user2].append(grad_vect[user2][i])
            # vectorizing the gradient
            #grad_vect[user2]=grad_vect[user2]+error[user2] # error feedback
            #array_one=np.ones((d,1))
            model_vector,layer_size=weight_vectorization_gen(w)
#             M=max(abs(model_vector))
#             mask=m_global
#             grad_vect_send[user2]=np.multiply(mask,grad_vect[user2])
            #error[user2]=grad_vect[user2]-grad_vect_send[user2]
           
            count=0
            grad_vect_send[user2]=grad_vect[user2]
            #norm.append(np.linalg.norm(grad_vect_send[user2],))
            #error[user2]=grad_vect[user2]-grad_vect_send[user2]
            grad_locals.append(grad_vect_send[user2])
            #store_grad[user2]=grad_vect_send[user2]
            diff_locals.append(copy.deepcopy(model_diff[user2]))
#             n=K*b*K*b
#             H = np.random.normal(loc=0, scale=np.sqrt(2)/2, size=(n, 2)).view(np.complex128)
#             H=real(H.reshape(K*b,K*b))
            #U, D, VT = np.linalg.svd(H)
            
            P_T=10000

            P_T=100000
            #cluster_gradient[cluster_no].append(H@(E1@grad_vect_send[user2]))
            #cluster_gradient[cluster_no].append(np.real(H@E1)@grad_vect_send[user2])
            
            #SNR=math.log10(M**2)
            
            Power[cluster_no].append(total_P)
        
        #Power2[cluster_no]=(sum(Power[cluster_no]))/len(cluster_block[cluster_no])
        P_limit=2000000000
        #alpha[cluster_no]=math.sqrt(P_limit/Power2[cluster_no])
        alpha[cluster_no]=100*100
        c=100*100
        grad_avg=FedAvg_gradient(grad_locals)
        #print(grad_avg)
        decoded[cluster_no]=grad_avg
        #enc=H@(E1@grad_avg)
        
        j=w_locals[0]
        feature_vector=grad_avg
        #print(len(feature_vector))
        #print(feature_vector)
        #fecture vector is dx1 vector after flattening all types of weight matrices and concatenaitng them
        increment=0 #keeps track of feature_vactor index
        #superposition=superposition+alpha[cluster_no]*encoder1[cluster_no]@feature_vector+num
    #received=superposition
    #print(len(superposition))
    #superposition=superposition +(1/c)*num*(1/min_norm)
    #print("total received Power")
    #print(sum((received)**2))
    #print("received min max")
    #print(max(abs(received)))
    #print(min(abs(received)))
    #m_global=np.zeros((d,1))
    recovered=[] 
    backup=[]
    del model_diff
    del grad_vect
    del grad_vect_send
    del prev
    del updated
    for cluster_no in range(args.cluster):
#         recovered.append(np.zeros((62006,1)))
#         backup.append(np.zeros((62006,1)))
        #print(w_glob[cluster_no])
        if(cluster_block[cluster_no]==[]):
            continue
        flat=[]
        
        count=0
        for i in range(16): # 4 layers in parameter
            flat.append([])
        increment=0
        count=0
        #print(decoded[cluster_no])
        #back=decoded[cluster_no]
        #recovered=np.zeros((d,1))
        index=0
        #print(decoded[cluster_no])
        w_glob_prev=copy.deepcopy(w_glob[cluster_no])
        for i in range(len(w_glob[cluster_no].keys())): # 4 layers in parameter
            flat.append([])

        for h in w_glob_prev.keys():
            s=list(w_glob[cluster_no][h].shape)
            if (len(s)==0):
                new=np.array(0)
                decoded[cluster_no]=np.delete(decoded[cluster_no],np.s_[0])
            else:
                z=np.prod(list(w_glob[cluster_no][h].shape))
                flat[count]=decoded[cluster_no][0:z] # taking out the vector for the specified layer
                decoded[cluster_no]=np.delete(decoded[cluster_no],np.s_[0:z])# deleting that vector from decoded after taking out
             
                new=flat[count].reshape(list(w_glob[cluster_no][h].shape)) #reshaping back to the marix
              
            w_glob[cluster_no][h]=torch.from_numpy(new) #converting the matrix to a tensor
            #print(w_glob[cluster_no][h].shape)
            count=count+1
    # update global weights
        
        global_diff = w_glob[cluster_no]
        w_glob[cluster_no]=FedAdd(w_glob_prev,global_diff)
        net_glob[cluster_no].load_state_dict(w_glob[cluster_no])
        #printing loss in each iteration
        for i in range(args.cluster):
            acc_test[i], loss_test[i] = test_acc(net_glob[cluster_no], test_data[i], args)
            acc_test_arr[i][cluster_no].append(acc_test[i])
            loss_test_arr[i][cluster_no].append(loss_test[i])
        for user in cluster_block[cluster_no]:
            acc,loss= test_acc(net_glob[cluster_no], sorted_test_data[user], args)
            noise_acc[user].append(acc)
            if(user==0)|(user==5)|(user==10)|(user==15)|(user==20):
                print(acc)
        #print(acc_test[cluster_no])
        #print(loss_test[cluster_no])
        #if iter % 1 ==0:
            #print('Round {:3d}, Average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_avg[cluster_no],acc_test[cluster_no]))
        #print(hist_)
        #print("users in cluster",cluster_no)
        #print(cluster_block[cluster_no])
        #print("Test accuracy of cluster",cluster_no)
        #print(acc_test[cluster_no])
    #print(loss_train)
    sample=sample+2

iteration number 0
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4, 14]
[10, 11, 12, 13]
[20, 21, 22, 23, 24]
tensor(50.)
tensor(50.)
tensor(50.0500)
tensor(50.)
tensor(50.)
iteration number 1
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
tensor(50.)
tensor(70.7000)
tensor(52.0500)
tensor(50.)
tensor(50.1500)
iteration number 2
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
tensor(50.)
tensor(71.7000)
tensor(70.1000)
tensor(54.5500)
tensor(68.)
iteration number 3
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
tensor(53.0500)
tensor(73.7500)
tensor(77.7500)
tensor(64.)
tensor(68.7500)
iteration number 4
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
tensor(53.)
tensor(75.4000)
tensor(77.6500)
tensor(65.0500)
tensor(68.6000)
iteration number 5
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4

tensor(74.6500)
tensor(87.8500)
tensor(86.6000)
tensor(78.9000)
tensor(82.7000)
iteration number 43
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
tensor(73.2500)
tensor(87.9500)
tensor(86.4000)
tensor(79.3000)
tensor(82.5500)
iteration number 44
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
tensor(75.1500)
tensor(87.9500)
tensor(86.5500)
tensor(77.8500)
tensor(82.1500)
iteration number 45
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
tensor(74.5000)
tensor(87.9000)
tensor(86.6500)
tensor(79.1000)
tensor(82.6500)
iteration number 46
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
tensor(74.9500)
tensor(87.5500)
tensor(86.6000)
tensor(79.1500)
tensor(83.2000)
iteration number 47
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
tensor(74.7000)
tensor(88

tensor(86.4500)
iteration number 85
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
tensor(76.3000)
tensor(89.8000)
tensor(88.8000)
tensor(80.6000)
tensor(86.4000)
iteration number 86
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
tensor(76.5000)
tensor(90.2000)
tensor(88.6500)
tensor(81.0500)
tensor(86.9000)
iteration number 87
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
tensor(75.4000)
tensor(90.2000)
tensor(89.1000)
tensor(81.2500)
tensor(86.4500)
iteration number 88
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
tensor(76.4500)
tensor(89.9500)
tensor(89.5000)
tensor(80.6500)
tensor(86.8500)
iteration number 89
[5, 6, 7, 8, 9]
[15, 16, 17, 18, 19]
[0, 1, 2, 3, 4]
[10, 11, 12, 13, 14]
[20, 21, 22, 23, 24]
tensor(75.8500)
tensor(89.8500)
tensor(88.8000)
tensor(80.5500)
tensor(86.5000)
iteration

KeyboardInterrupt: 

In [ ]:
print(type(num))
print(alpha[cluster_no])
print(num*(1/min_norm))

In [ ]:
a=[3, 9]
print(np.linalg.norm(a))

In [ ]:
print(len(encoder1[cluster_no][0]))

In [ ]:
print(sum(np.array(loss_locals)))

In [ ]:
print(cluster_gradient[cluster_no])

In [ ]:
user_acc=[]
for i in range(25):
    user_acc.append([])
for i in range(25):
    for j in noise_acc[i]:
        user_acc[i].append(float(j))
print(user_acc[19][60])

In [ ]:
Avg_acc_cluster0=[]
Avg_acc_cluster1=[]
Avg_acc_cluster2=[]
Avg_acc_cluster3=[]
Avg_acc_cluster4=[]
for i in range(1000):
    Avg_acc_cluster0.append((user_acc[0][i]+user_acc[1][i]+user_acc[2][i]+user_acc[3][i]+user_acc[4][i])/5)
    Avg_acc_cluster1.append((user_acc[5][i]+user_acc[6][i]+user_acc[7][i]+user_acc[8][i]+user_acc[9][i])/5)
    Avg_acc_cluster2.append((user_acc[10][i]+user_acc[11][i]+user_acc[12][i]+user_acc[13][i]+user_acc[14][i])/5)
    Avg_acc_cluster3.append((user_acc[15][i]+user_acc[16][i]+user_acc[17][i]+user_acc[18][i]+user_acc[19][i])/5)
    Avg_acc_cluster4.append((user_acc[20][i]+user_acc[21][i]+user_acc[22][i]+user_acc[23][i]+user_acc[24][i])/5)

In [ ]:
plt.plot(range(len(Avg_acc_cluster0)), Avg_acc_cluster0)
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 0 test accuracy')
plt.xlabel('iteration no.')
plt.show()
# plt.savefig('./save/fed_{}_{}_{}_C{}_iid{}.png'.format(args.dataset, args.model, args.epochs, args.frac, args.iid))
plt.plot(range(len(Avg_acc_cluster1)), Avg_acc_cluster1)
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 1 test accuracy')
plt.xlabel('iteration no.')
plt.show()
plt.plot(range(len(Avg_acc_cluster2)), Avg_acc_cluster2)
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 2 test accuracy')
plt.xlabel('iteration no.')
plt.show()
plt.plot(range(len(Avg_acc_cluster3)), Avg_acc_cluster3)
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 3 test accuracy')
plt.xlabel('iteration no.')
plt.show()
plt.plot(range(len(Avg_acc_cluster4)), Avg_acc_cluster4)
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 4 test accuracy')
plt.xlabel('iteration no.')
plt.show()

In [ ]:
print(Avg_acc_cluster0)
print("gap")
print(Avg_acc_cluster1)
print("gap")
print(Avg_acc_cluster2)
print("gap")
print(Avg_acc_cluster3)
print("gap")
print(Avg_acc_cluster4)
print("gap")

In [ ]:
#5000 antenna
Cluster_0_accuracy=[35.05999954342842, 50.0, 50.0, 66.25, 50.45000076293945, 71.19999694824219, 70.5999984741211, 74.0, 75.69999694824219, 75.75, 76.05000305175781, 76.3499984741211, 76.69999694824219, 77.1500015258789, 77.0, 76.80000305175781, 76.1500015258789, 76.0999984741211, 78.9000015258789, 79.05000305175781, 79.30000305175781, 79.69999694824219, 78.55000305175781, 80.5, 79.44999694824219, 80.05000305175781, 81.30000305175781, 81.55000305175781, 81.4000015258789, 82.19999694824219, 81.9000015258789, 82.1500015258789, 82.55000305175781, 83.0, 82.25, 82.94999694824219, 83.0999984741211, 83.55000305175781, 83.44999694824219, 83.69999694824219, 83.80000305175781, 83.80000305175781, 84.1500015258789, 84.55000305175781, 83.80000305175781, 84.19999694824219, 84.30000305175781, 84.5, 84.55000305175781, 85.0999984741211, 84.69999694824219, 84.94999694824219, 85.05000305175781, 84.5999984741211, 85.0999984741211, 85.44999694824219, 85.4000015258789, 84.69999694824219, 85.1500015258789, 85.4000015258789, 85.05000305175781, 85.05000305175781, 85.6500015258789, 85.05000305175781, 85.44999694824219, 85.3499984741211, 85.05000305175781, 85.80000305175781, 85.5999984741211, 85.8499984741211, 85.5999984741211, 85.80000305175781, 85.6500015258789, 85.25, 85.94999694824219, 85.8499984741211, 86.3499984741211, 86.0, 86.19999694824219, 86.19999694824219, 86.3499984741211, 86.30000305175781, 86.25, 86.19999694824219, 86.19999694824219, 86.25, 86.30000305175781, 86.5999984741211, 86.4000015258789, 86.55000305175781, 86.44999694824219, 86.9000015258789, 86.94999694824219, 87.0999984741211, 86.69999694824219, 87.19999694824219, 86.75, 86.8499984741211, 87.4000015258789, 87.3499984741211, 86.8499984741211, 86.8499984741211, 87.6500015258789, 87.25, 87.30000305175781, 87.19999694824219, 87.05000305175781, 87.6500015258789, 87.1500015258789, 87.19999694824219, 87.1500015258789, 87.69999694824219, 87.25, 87.5999984741211, 87.25, 87.80000305175781, 87.44999694824219, 87.5, 87.55000305175781, 87.5999984741211, 87.55000305175781, 87.80000305175781, 87.55000305175781, 87.80000305175781, 88.0, 87.5999984741211, 88.1500015258789, 87.94999694824219, 88.05000305175781, 88.19999694824219, 88.0, 88.05000305175781, 87.80000305175781, 88.0, 88.4000015258789, 88.44999694824219, 88.44999694824219, 88.4000015258789, 88.5, 88.75, 87.94999694824219, 88.6500015258789, 88.5999984741211, 88.3499984741211, 88.55000305175781, 88.80000305175781, 88.80000305175781, 88.69999694824219, 89.05000305175781, 88.6500015258789, 88.8499984741211, 88.69999694824219, 88.4000015258789, 88.5999984741211, 88.75, 89.0, 88.9000015258789, 88.69999694824219, 88.69999694824219, 89.0999984741211, 88.6500015258789, 88.5, 88.9000015258789, 88.5, 88.75, 89.0, 89.0999984741211, 88.8499984741211, 88.6500015258789, 88.9000015258789, 89.1500015258789, 89.30000305175781, 89.3499984741211, 88.80000305175781, 89.0999984741211, 88.94999694824219, 89.05000305175781, 89.05000305175781, 89.5, 89.30000305175781, 89.05000305175781, 87.69999694824219, 89.5, 89.30000305175781, 89.19999694824219, 89.5, 89.1500015258789, 89.0999984741211, 88.8499984741211, 89.05000305175781, 89.30000305175781, 89.30000305175781, 89.5, 89.19999694824219, 89.30000305175781, 89.3499984741211, 89.19999694824219, 89.19999694824219, 88.75, 89.30000305175781]
Cluster_1_accuracy=[50.0, 50.0, 50.0, 50.0, 52.599998474121094, 53.099998474121094, 50.349998474121094, 54.29999923706055, 54.25, 60.54999923706055, 54.29999923706055, 61.650001525878906, 61.75, 62.150001525878906, 61.79999923706055, 63.150001525878906, 62.849998474121094, 64.1500015258789, 65.5, 62.099998474121094, 66.0, 67.55000305175781, 66.6500015258789, 68.55000305175781, 66.25, 66.5, 69.05000305175781, 69.8499984741211, 70.05000305175781, 70.6500015258789, 70.05000305175781, 71.05000305175781, 70.8499984741211, 69.05000305175781, 71.6500015258789, 70.94999694824219, 71.8499984741211, 70.5, 70.05000305175781, 71.1500015258789, 72.19999694824219, 71.69999694824219, 71.25, 72.94999694824219, 71.9000015258789, 73.30000305175781, 72.75, 71.9000015258789, 73.30000305175781, 73.3499984741211, 73.1500015258789, 72.94999694824219, 73.55000305175781, 72.30000305175781, 72.75, 72.5999984741211, 73.80000305175781, 72.25, 73.30000305175781, 72.4000015258789, 73.0999984741211, 73.5, 73.55000305175781, 73.5999984741211, 73.6500015258789, 73.5, 73.55000305175781, 73.8499984741211, 73.1500015258789, 73.4000015258789, 74.0999984741211, 73.44999694824219, 73.3499984741211, 73.80000305175781, 73.8499984741211, 73.44999694824219, 73.69999694824219, 73.5999984741211, 73.75, 73.0, 73.94999694824219, 73.80000305175781, 73.8499984741211, 73.80000305175781, 73.9000015258789, 74.05000305175781, 73.6500015258789, 73.0999984741211, 73.94999694824219, 73.94999694824219, 74.30000305175781, 73.80000305175781, 74.05000305175781, 73.6500015258789, 73.80000305175781, 74.0999984741211, 74.19999694824219, 74.0, 74.05000305175781, 74.44999694824219, 74.1500015258789, 74.30000305175781, 73.5, 74.30000305175781, 74.25, 73.80000305175781, 73.94999694824219, 74.1500015258789, 73.94999694824219, 73.8499984741211, 73.75, 74.30000305175781, 74.0, 74.05000305175781, 74.05000305175781, 74.0999984741211, 74.3499984741211, 74.30000305175781, 74.6500015258789, 74.44999694824219, 73.8499984741211, 74.5999984741211, 74.0999984741211, 73.8499984741211, 74.5, 74.55000305175781, 74.6500015258789, 74.19999694824219, 74.9000015258789, 75.1500015258789, 74.9000015258789, 74.05000305175781, 74.3499984741211, 74.1500015258789, 74.75, 74.44999694824219, 74.44999694824219, 75.25, 74.8499984741211, 75.25, 74.05000305175781, 75.5, 74.0, 75.30000305175781, 74.80000305175781, 74.5, 74.6500015258789, 74.9000015258789, 74.44999694824219, 75.3499984741211, 75.0, 74.0999984741211, 74.5, 75.5, 75.0, 75.25, 75.1500015258789, 75.4000015258789, 75.1500015258789, 74.44999694824219, 74.9000015258789, 75.44999694824219, 75.4000015258789, 74.75, 74.5999984741211, 75.0, 75.19999694824219, 74.94999694824219, 75.0999984741211, 75.44999694824219, 74.80000305175781, 75.19999694824219, 75.55000305175781, 75.25, 75.25, 74.9000015258789, 74.8499984741211, 75.44999694824219, 75.5, 74.94999694824219, 75.19999694824219, 74.94999694824219, 75.0999984741211, 75.3499984741211, 75.1500015258789, 75.55000305175781, 75.0999984741211, 75.55000305175781, 75.25, 75.4000015258789, 74.94999694824219, 75.19999694824219, 75.0999984741211, 75.5999984741211, 75.0, 75.44999694824219, 75.0, 75.9000015258789, 75.5999984741211, 75.25]
Cluster_2_accuracy=[51.900001525878906, 59.0, 58.099998474121094, 50.0, 55.54999923706055, 50.45000076293945, 56.45000076293945, 62.349998474121094, 66.0999984741211, 60.5, 65.94999694824219, 65.0, 67.0999984741211, 68.0, 67.4000015258789, 68.5, 69.55000305175781, 67.5999984741211, 70.05000305175781, 70.6500015258789, 71.5999984741211, 71.80000305175781, 72.55000305175781, 72.25, 71.5, 73.80000305175781, 74.75, 74.1500015258789, 74.55000305175781, 74.3499984741211, 75.0999984741211, 76.0999984741211, 75.5999984741211, 76.30000305175781, 76.25, 77.5999984741211, 76.05000305175781, 77.75, 77.1500015258789, 77.55000305175781, 76.69999694824219, 78.0999984741211, 77.5, 77.8499984741211, 77.8499984741211, 78.30000305175781, 78.3499984741211, 78.25, 78.1500015258789, 78.19999694824219, 78.80000305175781, 78.3499984741211, 79.05000305175781, 78.44999694824219, 78.44999694824219, 78.80000305175781, 78.3499984741211, 78.30000305175781, 78.80000305175781, 79.1500015258789, 78.9000015258789, 79.0999984741211, 79.05000305175781, 79.0, 79.19999694824219, 78.0, 79.30000305175781, 79.5999984741211, 79.05000305175781, 80.19999694824219, 79.30000305175781, 79.25, 79.25, 79.6500015258789, 80.0, 80.3499984741211, 80.0999984741211, 80.1500015258789, 79.5999984741211, 80.19999694824219, 79.80000305175781, 80.4000015258789, 79.5, 80.6500015258789, 79.75, 79.8499984741211, 80.19999694824219, 81.0, 79.80000305175781, 79.9000015258789, 80.25, 80.55000305175781, 80.55000305175781, 81.0, 80.4000015258789, 80.3499984741211, 80.19999694824219, 80.9000015258789, 80.69999694824219, 81.0, 80.30000305175781, 81.0, 80.69999694824219, 80.3499984741211, 80.19999694824219, 80.75, 80.69999694824219, 80.94999694824219, 80.6500015258789, 80.8499984741211, 80.4000015258789, 80.94999694824219, 80.5999984741211, 81.0, 80.25, 80.94999694824219, 81.0999984741211, 81.1500015258789, 81.0, 80.94999694824219, 80.5999984741211, 81.05000305175781, 80.75, 80.94999694824219, 80.44999694824219, 81.19999694824219, 80.30000305175781, 81.5, 80.69999694824219, 81.0, 80.8499984741211, 81.19999694824219, 80.5, 81.0, 80.19999694824219, 80.9000015258789, 80.44999694824219, 81.4000015258789, 80.5999984741211, 81.0, 80.44999694824219, 80.9000015258789, 80.4000015258789, 80.8499984741211, 80.6500015258789, 80.9000015258789, 80.5999984741211, 81.0999984741211, 80.25, 80.94999694824219, 80.44999694824219, 80.9000015258789, 80.3499984741211, 81.0999984741211, 80.80000305175781, 81.05000305175781, 80.44999694824219, 81.19999694824219, 80.44999694824219, 81.3499984741211, 80.6500015258789, 81.0999984741211, 80.69999694824219, 81.0999984741211, 80.55000305175781, 81.30000305175781, 80.8499984741211, 81.4000015258789, 81.0, 81.44999694824219, 81.05000305175781, 81.1500015258789, 80.69999694824219, 81.5999984741211, 81.05000305175781, 81.4000015258789, 80.69999694824219, 81.44999694824219, 80.75, 81.25, 81.0, 81.30000305175781, 81.19999694824219, 81.4000015258789, 80.6500015258789, 81.5, 81.0999984741211, 82.0999984741211, 80.80000305175781, 81.55000305175781, 80.8499984741211, 81.4000015258789, 81.3499984741211, 81.8499984741211, 81.19999694824219, 81.80000305175781, 81.4000015258789, 82.1500015258789, 81.0999984741211, 82.19999694824219]
Cluster_3_accuracy=[50.29999923706055, 50.0, 50.0, 50.0, 53.5, 68.25, 66.3499984741211, 67.75, 70.6500015258789, 71.19999694824219, 74.25, 72.75, 73.25, 73.19999694824219, 76.05000305175781, 76.44999694824219, 75.44999694824219, 75.6500015258789, 78.5999984741211, 74.19999694824219, 78.55000305175781, 78.6500015258789, 80.1500015258789, 79.69999694824219, 80.4000015258789, 80.0999984741211, 80.8499984741211, 80.0, 81.25, 82.1500015258789, 82.69999694824219, 82.75, 82.5999984741211, 82.94999694824219, 83.3499984741211, 83.3499984741211, 83.75, 84.30000305175781, 84.0999984741211, 84.25, 84.5999984741211, 84.55000305175781, 85.55000305175781, 85.0999984741211, 85.6500015258789, 85.5, 86.19999694824219, 86.1500015258789, 86.69999694824219, 86.05000305175781, 86.6500015258789, 86.30000305175781, 87.1500015258789, 86.4000015258789, 87.0999984741211, 86.8499984741211, 87.5, 87.4000015258789, 87.3499984741211, 87.3499984741211, 87.69999694824219, 87.4000015258789, 87.80000305175781, 87.6500015258789, 88.1500015258789, 88.0, 88.1500015258789, 88.0999984741211, 88.25, 88.0999984741211, 88.0999984741211, 88.19999694824219, 88.1500015258789, 88.30000305175781, 88.05000305175781, 88.05000305175781, 88.19999694824219, 88.30000305175781, 88.0999984741211, 88.94999694824219, 88.44999694824219, 88.80000305175781, 88.75, 88.80000305175781, 88.5999984741211, 88.80000305175781, 88.6500015258789, 89.0999984741211, 89.0, 88.75, 89.0, 89.05000305175781, 89.19999694824219, 89.0, 89.4000015258789, 89.05000305175781, 89.19999694824219, 89.19999694824219, 89.0, 89.44999694824219, 89.5999984741211, 89.1500015258789, 89.4000015258789, 89.44999694824219, 89.44999694824219, 89.5999984741211, 89.3499984741211, 89.3499984741211, 89.5, 89.55000305175781, 89.44999694824219, 89.69999694824219, 89.44999694824219, 89.44999694824219, 89.6500015258789, 89.4000015258789, 89.44999694824219, 89.6500015258789, 89.55000305175781, 89.55000305175781, 90.05000305175781, 89.9000015258789, 89.75, 89.75, 89.44999694824219, 89.80000305175781, 89.6500015258789, 89.75, 89.6500015258789, 89.8499984741211, 90.1500015258789, 89.80000305175781, 89.30000305175781, 89.80000305175781, 89.44999694824219, 89.94999694824219, 90.0, 89.8499984741211, 90.0, 90.1500015258789, 89.75, 90.05000305175781, 90.05000305175781, 90.05000305175781, 89.8499984741211, 90.1500015258789, 90.05000305175781, 89.9000015258789, 90.0, 90.5, 89.80000305175781, 90.30000305175781, 90.19999694824219, 90.30000305175781, 90.25, 90.5, 90.0, 90.6500015258789, 90.05000305175781, 90.1500015258789, 89.80000305175781, 90.44999694824219, 90.30000305175781, 90.30000305175781, 90.0999984741211, 90.55000305175781, 90.19999694824219, 90.19999694824219, 89.9000015258789, 90.75, 90.0, 91.0999984741211, 90.0999984741211, 90.8499984741211, 90.19999694824219, 91.0, 90.3499984741211, 91.05000305175781, 90.25, 90.94999694824219, 90.44999694824219, 90.80000305175781, 90.3499984741211, 91.1500015258789, 90.5999984741211, 90.9000015258789, 90.0, 91.1500015258789, 90.0, 91.0999984741211, 90.1500015258789, 90.55000305175781, 90.1500015258789, 91.25, 90.0999984741211, 91.0999984741211, 90.4000015258789, 91.25, 89.94999694824219, 91.19999694824219]
Cluster_4_accuracy=[50.0, 50.0, 50.0, 50.0, 50.5, 66.19999694824219, 67.30000305175781, 66.0999984741211, 68.94999694824219, 64.80000305175781, 69.4000015258789, 70.19999694824219, 69.8499984741211, 70.4000015258789, 71.5, 71.8499984741211, 72.25, 72.8499984741211, 72.80000305175781, 73.55000305175781, 73.80000305175781, 74.44999694824219, 74.6500015258789, 75.5, 75.0, 76.44999694824219, 76.30000305175781, 77.30000305175781, 77.44999694824219, 78.30000305175781, 78.25, 78.94999694824219, 78.9000015258789, 79.0, 79.1500015258789, 79.4000015258789, 79.4000015258789, 80.19999694824219, 80.5999984741211, 80.6500015258789, 80.5999984741211, 81.1500015258789, 81.0, 81.0999984741211, 80.94999694824219, 81.4000015258789, 80.5999984741211, 81.3499984741211, 81.6500015258789, 81.69999694824219, 82.0, 82.5999984741211, 82.25, 82.5999984741211, 82.44999694824219, 82.4000015258789, 82.69999694824219, 83.4000015258789, 82.44999694824219, 83.19999694824219, 83.44999694824219, 83.75, 83.75, 83.94999694824219, 83.8499984741211, 84.1500015258789, 84.0999984741211, 84.0999984741211, 84.4000015258789, 84.75, 84.4000015258789, 84.6500015258789, 84.30000305175781, 85.0, 84.55000305175781, 84.8499984741211, 85.0999984741211, 85.05000305175781, 84.94999694824219, 85.25, 85.3499984741211, 85.55000305175781, 85.6500015258789, 85.75, 85.55000305175781, 86.05000305175781, 85.69999694824219, 85.44999694824219, 85.5, 86.19999694824219, 86.1500015258789, 86.0, 86.05000305175781, 86.25, 86.3499984741211, 86.3499984741211, 86.55000305175781, 86.30000305175781, 86.19999694824219, 86.6500015258789, 86.30000305175781, 86.75, 86.6500015258789, 86.3499984741211, 86.44999694824219, 86.5999984741211, 86.0999984741211, 86.75, 86.5, 86.69999694824219, 86.5999984741211, 86.80000305175781, 86.80000305175781, 86.80000305175781, 86.6500015258789, 86.9000015258789, 86.80000305175781, 86.80000305175781, 87.1500015258789, 86.75, 86.8499984741211, 86.94999694824219, 86.9000015258789, 86.9000015258789, 86.75, 87.44999694824219, 87.3499984741211, 87.5, 87.05000305175781, 87.3499984741211, 87.0, 87.25, 87.1500015258789, 87.25, 87.0999984741211, 87.5, 87.0999984741211, 87.25, 87.19999694824219, 87.6500015258789, 87.4000015258789, 87.5, 87.1500015258789, 87.5999984741211, 87.4000015258789, 87.6500015258789, 87.55000305175781, 87.3499984741211, 87.3499984741211, 87.0999984741211, 87.44999694824219, 87.8499984741211, 87.4000015258789, 87.5999984741211, 87.44999694824219, 87.25, 87.55000305175781, 87.69999694824219, 87.5999984741211, 87.55000305175781, 87.1500015258789, 87.75, 87.69999694824219, 87.5999984741211, 87.44999694824219, 87.69999694824219, 87.44999694824219, 87.5999984741211, 87.55000305175781, 88.05000305175781, 87.75, 87.8499984741211, 87.80000305175781, 87.75, 88.05000305175781, 88.05000305175781, 87.80000305175781, 87.80000305175781, 87.80000305175781, 88.25, 88.4000015258789, 88.1500015258789, 88.19999694824219, 87.94999694824219, 88.25, 88.05000305175781, 88.1500015258789, 88.25, 87.94999694824219, 88.30000305175781, 88.0999984741211, 88.0999984741211, 88.4000015258789, 88.3499984741211, 88.3499984741211, 88.3499984741211, 88.44999694824219, 88.55000305175781, 88.55000305175781, 88.6500015258789]

#100 antenna lr=0.001
Cluster_0_accuracy_100=[50.0, 50.0, 50.099998474121094, 51.79999923706055, 52.150001525878906, 61.79999923706055, 73.0999984741211, 50.20000076293945, 68.4000015258789, 73.75, 74.1500015258789, 76.25, 72.55000305175781, 76.9000015258789, 69.8499984741211, 64.9000015258789, 74.8499984741211, 60.349998474121094, 75.80000305175781, 72.5999984741211, 67.05000305175781, 64.25, 76.44999694824219, 71.5999984741211, 72.44999694824219, 75.25, 64.75, 73.5, 70.1500015258789, 65.9000015258789, 74.6500015258789, 73.5999984741211, 71.55000305175781, 72.55000305175781, 75.44999694824219, 72.94999694824219, 66.25, 64.0999984741211, 72.6500015258789, 67.55000305175781, 74.05000305175781, 74.9000015258789, 74.6500015258789, 75.4000015258789, 75.80000305175781, 74.6500015258789, 76.6500015258789, 67.44999694824219, 70.8499984741211, 72.94999694824219, 77.0999984741211, 77.19999694824219, 76.44999694824219, 77.0999984741211, 74.55000305175781, 71.75, 76.5, 77.19999694824219, 77.05000305175781, 77.30000305175781, 77.5999984741211, 75.05000305175781, 71.4000015258789, 73.44999694824219, 70.80000305175781, 78.4000015258789, 75.05000305175781, 76.55000305175781, 70.8499984741211, 78.19999694824219, 75.0, 66.8499984741211, 67.55000305175781, 77.05000305175781, 76.1500015258789, 78.4000015258789, 71.75, 74.5999984741211, 73.75, 75.75, 72.94999694824219, 73.8499984741211, 77.55000305175781, 70.5999984741211, 67.0, 78.55000305175781, 78.5999984741211, 78.3499984741211, 71.19999694824219, 77.44999694824219, 75.44999694824219, 71.44999694824219, 72.55000305175781, 74.0, 76.19999694824219, 69.9000015258789, 77.5999984741211, 73.3499984741211, 76.3499984741211, 75.19999694824219, 75.25, 73.19999694824219, 69.94999694824219, 71.0, 72.94999694824219, 73.19999694824219, 66.25, 73.94999694824219, 75.19999694824219, 76.44999694824219, 76.8499984741211, 71.5999984741211, 73.6500015258789, 71.80000305175781, 66.5, 74.1500015258789, 74.30000305175781, 74.69999694824219, 71.4000015258789, 70.8499984741211, 74.5, 71.80000305175781, 71.5, 70.05000305175781, 73.0, 73.69999694824219, 74.5, 71.75, 73.4000015258789, 75.25, 72.94999694824219, 72.5999984741211, 74.4000015258789, 72.80000305175781, 73.0999984741211, 71.6500015258789, 73.30000305175781, 72.1500015258789, 74.5999984741211, 72.0999984741211, 74.25, 75.75, 76.05000305175781, 74.3499984741211, 77.05000305175781, 74.0, 74.05000305175781, 74.55000305175781, 71.80000305175781, 73.05000305175781, 71.3499984741211, 74.6500015258789, 72.0, 73.80000305175781, 76.6500015258789, 75.6500015258789, 77.1500015258789, 65.9000015258789, 77.1500015258789, 76.5999984741211, 75.25, 76.19999694824219, 76.75, 75.6500015258789, 76.5999984741211, 73.1500015258789, 75.69999694824219, 76.69999694824219, 72.69999694824219, 75.1500015258789, 75.3499984741211, 74.05000305175781, 73.05000305175781, 73.5, 74.3499984741211, 73.69999694824219, 73.94999694824219, 74.30000305175781, 74.5999984741211, 73.75, 74.69999694824219, 73.4000015258789, 72.19999694824219, 74.19999694824219, 73.5, 75.5999984741211, 70.19999694824219, 75.1500015258789, 74.19999694824219, 73.1500015258789, 74.55000305175781, 73.4000015258789, 73.3499984741211, 76.5999984741211, 77.05000305175781, 75.0999984741211, 74.4000015258789, 77.1500015258789, 77.0999984741211, 75.8499984741211]
Cluster_1_accuracy_100=[47.95000076293945, 50.0, 49.45000076293945, 50.650001525878906, 51.150001525878906, 53.75, 58.849998474121094, 61.75, 55.849998474121094, 58.75, 57.20000076293945, 58.900001525878906, 62.150001525878906, 65.55000305175781, 61.70000076293945, 60.400001525878906, 63.29999923706055, 64.5999984741211, 65.0, 65.6500015258789, 64.05000305175781, 67.55000305175781, 62.20000076293945, 61.75, 66.69999694824219, 65.94999694824219, 66.3499984741211, 62.5, 65.9000015258789, 66.25, 67.05000305175781, 64.5999984741211, 62.45000076293945, 63.95000076293945, 64.30000305175781, 63.29999923706055, 62.0, 62.099998474121094, 61.849998474121094, 63.75, 61.599998474121094, 63.150001525878906, 65.05000305175781, 63.70000076293945, 65.5999984741211, 61.54999923706055, 67.8499984741211, 66.5, 65.8499984741211, 67.19999694824219, 67.05000305175781, 56.29999923706055, 62.599998474121094, 61.900001525878906, 63.20000076293945, 57.099998474121094, 53.349998474121094, 53.400001525878906, 60.349998474121094, 57.0, 59.79999923706055, 61.75, 58.0, 63.099998474121094, 60.95000076293945, 61.599998474121094, 58.599998474121094, 63.900001525878906, 63.0, 63.900001525878906, 59.0, 61.04999923706055, 63.79999923706055, 57.900001525878906, 61.349998474121094, 60.75, 64.5999984741211, 64.69999694824219, 58.5, 65.75, 64.80000305175781, 63.5, 62.25, 64.25, 62.900001525878906, 63.900001525878906, 63.400001525878906, 64.6500015258789, 62.400001525878906, 62.75, 65.4000015258789, 64.0999984741211, 65.4000015258789, 58.900001525878906, 67.05000305175781, 67.4000015258789, 67.55000305175781, 64.75, 63.54999923706055, 62.79999923706055, 64.05000305175781, 62.25, 59.5, 62.849998474121094, 64.0999984741211, 61.75, 59.95000076293945, 62.29999923706055, 64.0999984741211, 64.30000305175781, 62.0, 63.099998474121094, 61.45000076293945, 62.5, 66.0, 65.44999694824219, 65.1500015258789, 66.1500015258789, 64.19999694824219, 65.1500015258789, 63.25, 65.05000305175781, 60.70000076293945, 59.45000076293945, 62.099998474121094, 64.5, 63.75, 65.5, 65.25, 63.900001525878906, 64.5, 62.54999923706055, 63.70000076293945, 63.650001525878906, 63.54999923706055, 64.05000305175781, 63.54999923706055, 61.04999923706055, 61.349998474121094, 62.75, 65.5, 65.3499984741211, 63.95000076293945, 65.25, 63.5, 60.04999923706055, 62.70000076293945, 60.849998474121094, 61.0, 56.20000076293945, 59.900001525878906, 61.0, 61.25, 63.099998474121094, 62.150001525878906, 62.849998474121094, 62.349998474121094, 62.70000076293945, 61.849998474121094, 63.400001525878906, 61.25, 60.04999923706055, 63.900001525878906, 61.400001525878906, 57.650001525878906, 65.1500015258789, 62.95000076293945, 62.900001525878906, 62.650001525878906, 62.29999923706055, 62.400001525878906, 61.900001525878906, 58.349998474121094, 61.29999923706055, 62.25, 62.0, 63.5, 63.599998474121094, 59.79999923706055, 61.75, 62.150001525878906, 62.29999923706055, 61.95000076293945, 62.25, 62.400001525878906, 62.349998474121094, 62.099998474121094, 60.0, 61.599998474121094, 60.20000076293945, 58.75, 60.150001525878906, 60.45000076293945, 59.29999923706055, 60.5, 60.849998474121094, 60.900001525878906, 61.04999923706055, 61.20000076293945, 63.29999923706055]
Cluster_2_accuracy_100=[21.849999237060548, 50.20000076293945, 48.70000076293945, 51.29999923706055, 50.5, 52.70000076293945, 50.25, 50.349998474121094, 63.45000076293945, 61.79999923706055, 51.599998474121094, 61.5, 51.04999923706055, 67.30000305175781, 50.20000076293945, 58.75, 63.79999923706055, 59.900001525878906, 55.75, 65.94999694824219, 58.900001525878906, 64.55000305175781, 59.400001525878906, 64.75, 65.19999694824219, 66.55000305175781, 57.29999923706055, 54.29999923706055, 62.650001525878906, 53.95000076293945, 60.79999923706055, 65.6500015258789, 59.650001525878906, 67.5, 60.45000076293945, 56.349998474121094, 63.45000076293945, 66.3499984741211, 66.0, 65.44999694824219, 58.29999923706055, 60.650001525878906, 66.1500015258789, 61.04999923706055, 60.099998474121094, 60.75, 63.70000076293945, 56.150001525878906, 51.95000076293945, 57.79999923706055, 61.75, 64.80000305175781, 56.75, 56.5, 59.650001525878906, 52.849998474121094, 51.54999923706055, 60.400001525878906, 62.849998474121094, 59.349998474121094, 58.849998474121094, 61.95000076293945, 62.95000076293945, 54.849998474121094, 58.95000076293945, 64.80000305175781, 59.25, 61.20000076293945, 55.45000076293945, 59.54999923706055, 58.70000076293945, 62.25, 53.900001525878906, 62.95000076293945, 63.400001525878906, 60.650001525878906, 64.5, 65.8499984741211, 60.900001525878906, 66.0, 61.95000076293945, 62.70000076293945, 63.54999923706055, 64.44999694824219, 65.30000305175781, 65.5, 63.20000076293945, 65.0999984741211, 62.099998474121094, 58.20000076293945, 61.29999923706055, 61.150001525878906, 62.150001525878906, 62.75, 64.3499984741211, 62.5, 63.20000076293945, 64.0999984741211, 67.55000305175781, 57.04999923706055, 59.04999923706055, 68.4000015258789, 60.150001525878906, 63.20000076293945, 63.20000076293945, 60.5, 63.79999923706055, 66.69999694824219, 53.349998474121094, 61.349998474121094, 60.900001525878906, 61.70000076293945, 62.0, 63.04999923706055, 64.94999694824219, 62.70000076293945, 62.599998474121094, 63.29999923706055, 64.0, 63.400001525878906, 59.650001525878906, 56.29999923706055, 63.0, 63.54999923706055, 62.20000076293945, 63.400001525878906, 53.25, 58.900001525878906, 56.150001525878906, 58.25, 63.650001525878906, 60.599998474121094, 59.75, 60.25, 57.650001525878906, 62.849998474121094, 57.79999923706055, 59.45000076293945, 60.849998474121094, 62.400001525878906, 60.849998474121094, 61.150001525878906, 63.099998474121094, 62.29999923706055, 57.900001525878906, 62.54999923706055, 60.150001525878906, 61.849998474121094, 62.0, 62.04999923706055, 62.400001525878906, 63.150001525878906, 59.400001525878906, 55.900001525878906, 55.650001525878906, 51.54999923706055, 57.04999923706055, 60.75, 61.29999923706055, 62.04999923706055, 60.099998474121094, 56.849998474121094, 60.400001525878906, 64.75, 63.849998474121094, 60.849998474121094, 62.20000076293945, 59.79999923706055, 58.349998474121094, 60.900001525878906, 63.20000076293945, 62.29999923706055, 58.79999923706055, 61.25, 61.349998474121094, 62.900001525878906, 61.150001525878906, 63.04999923706055, 59.150001525878906, 63.900001525878906, 61.54999923706055, 60.95000076293945, 56.79999923706055, 61.75, 63.400001525878906, 64.44999694824219, 63.349998474121094, 63.45000076293945, 62.099998474121094, 64.05000305175781, 60.349998474121094, 62.5, 59.75, 60.099998474121094, 59.75, 53.79999923706055, 55.650001525878906, 53.45000076293945, 55.70000076293945, 55.70000076293945]
Cluster_3_accuracy_100=[31.31999939084053, 49.95000076293945, 51.95000076293945, 64.75, 56.900001525878906, 61.29999923706055, 50.400001525878906, 51.45000076293945, 58.70000076293945, 64.94999694824219, 73.6500015258789, 62.95000076293945, 55.54999923706055, 76.5, 73.75, 62.849998474121094, 72.8499984741211, 55.5, 73.8499984741211, 69.3499984741211, 75.0, 66.69999694824219, 72.5, 76.55000305175781, 79.0, 78.4000015258789, 72.19999694824219, 78.19999694824219, 68.19999694824219, 77.69999694824219, 78.9000015258789, 73.0, 77.25, 78.44999694824219, 62.70000076293945, 65.3499984741211, 77.0999984741211, 74.19999694824219, 76.94999694824219, 78.5999984741211, 66.05000305175781, 72.3499984741211, 76.1500015258789, 75.1500015258789, 67.3499984741211, 53.20000076293945, 67.19999694824219, 70.75, 76.1500015258789, 68.69999694824219, 73.1500015258789, 73.55000305175781, 72.1500015258789, 63.349998474121094, 77.5, 79.6500015258789, 62.04999923706055, 66.05000305175781, 57.04999923706055, 70.0, 66.6500015258789, 53.650001525878906, 76.3499984741211, 60.29999923706055, 71.5, 69.8499984741211, 70.75, 63.400001525878906, 74.0999984741211, 76.0, 72.30000305175781, 71.0999984741211, 72.9000015258789, 75.05000305175781, 71.94999694824219, 70.9000015258789, 71.80000305175781, 71.75, 71.80000305175781, 76.9000015258789, 70.5, 74.19999694824219, 64.44999694824219, 72.80000305175781, 74.55000305175781, 75.0999984741211, 75.8499984741211, 72.4000015258789, 73.80000305175781, 75.5999984741211, 73.44999694824219, 77.8499984741211, 71.69999694824219, 76.0999984741211, 75.3499984741211, 75.8499984741211, 75.1500015258789, 78.5999984741211, 75.30000305175781, 79.0999984741211, 73.94999694824219, 75.5, 76.30000305175781, 76.4000015258789, 74.5, 74.3499984741211, 77.80000305175781, 77.94999694824219, 77.8499984741211, 78.9000015258789, 76.80000305175781, 77.8499984741211, 73.0999984741211, 73.6500015258789, 75.80000305175781, 77.05000305175781, 76.0999984741211, 75.05000305175781, 77.25, 75.75, 76.30000305175781, 76.6500015258789, 76.75, 77.5, 76.05000305175781, 76.05000305175781, 77.75, 71.44999694824219, 76.75, 73.0, 77.94999694824219, 74.25, 73.80000305175781, 71.55000305175781, 76.69999694824219, 76.69999694824219, 78.6500015258789, 77.44999694824219, 76.55000305175781, 78.1500015258789, 74.19999694824219, 76.3499984741211, 65.1500015258789, 70.8499984741211, 75.44999694824219, 74.19999694824219, 74.30000305175781, 72.25, 71.30000305175781, 73.5999984741211, 74.25, 77.69999694824219, 73.8499984741211, 77.4000015258789, 76.44999694824219, 62.25, 72.1500015258789, 76.75, 75.8499984741211, 76.9000015258789, 77.05000305175781, 77.05000305175781, 76.4000015258789, 74.9000015258789, 76.69999694824219, 75.25, 75.5, 70.4000015258789, 73.9000015258789, 72.6500015258789, 77.55000305175781, 76.0999984741211, 74.44999694824219, 68.3499984741211, 74.25, 76.1500015258789, 75.94999694824219, 76.0, 73.80000305175781, 75.9000015258789, 74.05000305175781, 74.55000305175781, 73.1500015258789, 72.30000305175781, 70.3499984741211, 74.25, 75.55000305175781, 75.19999694824219, 75.30000305175781, 69.5999984741211, 73.69999694824219, 75.19999694824219, 73.30000305175781, 74.4000015258789, 70.69999694824219, 71.80000305175781, 72.25, 73.05000305175781, 74.6500015258789, 73.6500015258789]
Cluster_4_accuracy_100=[50.0, 50.0, 49.95000076293945, 48.95000076293945, 56.25, 65.5999984741211, 57.20000076293945, 67.9000015258789, 59.95000076293945, 65.94999694824219, 64.0, 53.5, 57.349998474121094, 66.44999694824219, 65.3499984741211, 55.79999923706055, 62.900001525878906, 55.5, 51.29999923706055, 53.5, 65.94999694824219, 52.0, 67.94999694824219, 68.3499984741211, 65.55000305175781, 69.25, 60.099998474121094, 63.849998474121094, 63.25, 68.5, 71.6500015258789, 68.1500015258789, 67.25, 64.30000305175781, 70.4000015258789, 69.55000305175781, 58.900001525878906, 59.650001525878906, 71.55000305175781, 64.44999694824219, 63.349998474121094, 65.1500015258789, 66.0999984741211, 70.05000305175781, 71.1500015258789, 70.69999694824219, 71.55000305175781, 71.0, 71.3499984741211, 67.75, 73.3499984741211, 64.6500015258789, 66.30000305175781, 65.05000305175781, 68.5999984741211, 60.599998474121094, 66.25, 63.04999923706055, 67.3499984741211, 67.55000305175781, 65.80000305175781, 63.25, 67.94999694824219, 63.20000076293945, 67.69999694824219, 62.45000076293945, 65.1500015258789, 65.1500015258789, 57.70000076293945, 57.04999923706055, 63.20000076293945, 66.0999984741211, 58.849998474121094, 67.0, 68.55000305175781, 67.0, 64.80000305175781, 68.75, 64.8499984741211, 64.25, 68.94999694824219, 66.3499984741211, 66.4000015258789, 67.8499984741211, 64.6500015258789, 62.25, 67.44999694824219, 60.75, 66.4000015258789, 63.79999923706055, 67.9000015258789, 65.94999694824219, 68.3499984741211, 65.44999694824219, 68.0999984741211, 68.69999694824219, 66.75, 67.80000305175781, 64.55000305175781, 68.5, 65.44999694824219, 62.150001525878906, 65.25, 66.0999984741211, 68.19999694824219, 66.9000015258789, 59.599998474121094, 64.69999694824219, 67.19999694824219, 59.650001525878906, 64.75, 62.70000076293945, 64.6500015258789, 64.19999694824219, 65.0, 65.25, 65.25, 65.8499984741211, 66.25, 66.25, 66.30000305175781, 64.94999694824219, 64.80000305175781, 66.55000305175781, 68.05000305175781, 66.75, 68.75, 66.5999984741211, 65.69999694824219, 67.1500015258789, 66.0999984741211, 63.849998474121094, 66.75, 66.25, 64.8499984741211, 66.19999694824219, 63.04999923706055, 62.5, 67.05000305175781, 66.3499984741211, 68.05000305175781, 65.8499984741211, 63.599998474121094, 63.45000076293945, 62.25, 66.5999984741211, 65.3499984741211, 63.349998474121094, 62.20000076293945, 62.400001525878906, 62.79999923706055, 61.150001525878906, 64.3499984741211, 63.900001525878906, 66.4000015258789, 68.1500015258789, 64.69999694824219, 63.95000076293945, 67.75, 65.5999984741211, 64.05000305175781, 63.95000076293945, 64.25, 61.900001525878906, 70.8499984741211, 65.05000305175781, 70.55000305175781, 68.3499984741211, 71.05000305175781, 69.4000015258789, 66.69999694824219, 65.94999694824219, 69.75, 66.8499984741211, 69.44999694824219, 67.8499984741211, 66.75, 69.1500015258789, 63.29999923706055, 69.6500015258789, 71.4000015258789, 70.25, 66.5999984741211, 69.69999694824219, 71.5, 71.0999984741211, 70.80000305175781, 69.5, 67.05000305175781, 70.5999984741211, 69.44999694824219, 69.1500015258789, 72.25, 71.1500015258789, 69.94999694824219, 68.1500015258789, 67.1500015258789, 67.25, 67.8499984741211, 60.650001525878906]

#50 antenna lr =0.00001
Cluster_0_accuracy_50=[44.20000076293945, 44.70000076293945, 48.150001525878906, 49.95000076293945, 50.45000076293945, 51.900001525878906, 56.79999923706055, 53.75, 50.0, 58.150001525878906, 54.25, 59.650001525878906, 54.45000076293945, 58.20000076293945, 63.45000076293945, 57.0, 50.0, 55.5, 52.04999923706055, 53.099998474121094, 50.5, 52.0, 50.04999923706055, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 54.25, 53.900001525878906, 50.099998474121094, 50.349998474121094, 55.29999923706055, 56.75, 66.25, 73.25, 69.5999984741211, 70.80000305175781, 73.75, 72.9000015258789, 69.6500015258789, 75.0999984741211, 70.75, 74.19999694824219, 70.0999984741211, 74.3499984741211, 75.0999984741211, 75.05000305175781, 74.44999694824219, 75.75, 73.9000015258789, 74.44999694824219, 75.75, 74.5999984741211, 74.30000305175781, 75.80000305175781, 74.55000305175781, 74.5, 73.94999694824219, 75.75, 75.0999984741211, 76.5999984741211, 76.55000305175781, 76.0999984741211, 76.75, 75.80000305175781, 75.19999694824219, 76.6500015258789, 76.80000305175781, 76.44999694824219, 76.6500015258789, 76.5999984741211, 75.5, 76.5999984741211, 75.4000015258789, 75.55000305175781, 76.0999984741211, 74.9000015258789, 76.75, 76.8499984741211, 75.19999694824219, 76.30000305175781, 77.1500015258789, 76.69999694824219, 76.44999694824219, 76.75, 76.44999694824219, 76.75, 76.8499984741211, 76.6500015258789, 75.94999694824219, 75.8499984741211, 77.3499984741211, 76.3499984741211, 76.80000305175781, 76.3499984741211, 77.5, 77.3499984741211, 76.9000015258789, 76.75, 78.25, 76.30000305175781, 77.1500015258789, 77.80000305175781, 76.5999984741211, 76.75, 77.19999694824219, 76.75, 77.30000305175781, 77.69999694824219, 78.5999984741211, 78.69999694824219, 78.6500015258789, 78.75, 77.4000015258789, 78.0, 78.9000015258789, 76.9000015258789, 77.0999984741211, 78.4000015258789, 79.4000015258789, 78.30000305175781, 79.3499984741211, 79.05000305175781, 79.3499984741211, 79.19999694824219, 77.4000015258789, 79.25, 78.19999694824219, 78.3499984741211, 78.4000015258789, 79.44999694824219, 79.44999694824219, 77.80000305175781, 77.3499984741211, 78.44999694824219, 77.9000015258789, 77.8499984741211, 78.80000305175781, 78.25, 77.94999694824219, 79.4000015258789, 78.3499984741211, 78.5999984741211, 78.69999694824219, 78.9000015258789, 79.4000015258789, 79.4000015258789, 79.44999694824219, 79.8499984741211, 79.5, 80.69999694824219, 80.05000305175781, 79.5, 80.3499984741211, 80.0, 80.55000305175781, 80.3499984741211, 80.5, 79.80000305175781, 80.5, 79.6500015258789, 81.0, 81.0999984741211, 80.94999694824219, 80.19999694824219, 81.0, 80.80000305175781, 79.75, 80.4000015258789, 80.3499984741211, 80.19999694824219, 80.94999694824219, 80.30000305175781, 81.0999984741211, 80.9000015258789, 81.0999984741211, 80.0, 80.8499984741211, 81.25, 80.80000305175781, 80.94999694824219, 80.69999694824219, 80.4000015258789, 80.6500015258789, 80.55000305175781, 80.44999694824219, 81.0999984741211, 81.30000305175781, 80.8499984741211, 81.0999984741211, 81.25, 80.19999694824219, 81.55000305175781, 81.80000305175781, 81.05000305175781, 81.8499984741211, 80.8499984741211, 82.05000305175781, 82.0999984741211, 81.80000305175781, 81.25, 82.0, 82.05000305175781, 81.25, 81.6500015258789, 81.0999984741211, 81.19999694824219, 80.9000015258789, 81.94999694824219, 81.5, 81.69999694824219, 81.8499984741211, 81.44999694824219, 82.19999694824219, 82.5, 82.69999694824219, 81.69999694824219, 82.9000015258789, 80.80000305175781, 82.19999694824219, 81.94999694824219, 82.19999694824219, 82.5, 82.19999694824219, 82.69999694824219, 82.30000305175781, 82.0999984741211, 82.6500015258789, 82.3499984741211, 82.3499984741211, 82.5999984741211, 82.5, 82.94999694824219, 83.19999694824219, 82.8499984741211, 81.3499984741211, 83.1500015258789, 82.1500015258789, 82.25, 82.4000015258789, 83.1500015258789, 83.0999984741211, 82.80000305175781, 82.80000305175781, 82.8499984741211, 83.19999694824219, 83.0, 82.9000015258789, 82.94999694824219, 82.80000305175781, 82.75, 83.3499984741211, 83.44999694824219, 83.4000015258789, 82.8499984741211, 82.8499984741211, 81.5999984741211, 82.80000305175781, 83.44999694824219, 83.3499984741211, 83.30000305175781, 83.4000015258789, 83.4000015258789, 82.0, 81.5999984741211, 83.25, 83.44999694824219, 83.4000015258789, 83.30000305175781, 83.4000015258789, 82.75, 83.0, 83.05000305175781, 82.44999694824219, 82.94999694824219, 83.30000305175781, 83.9000015258789, 83.5999984741211, 83.9000015258789, 83.8499984741211, 83.4000015258789, 83.75, 83.05000305175781, 83.4000015258789, 83.69999694824219, 83.8499984741211, 83.3499984741211, 82.9000015258789, 83.6500015258789, 83.5, 83.05000305175781, 82.94999694824219, 83.19999694824219, 83.4000015258789, 83.44999694824219, 82.44999694824219, 82.30000305175781, 83.3499984741211, 83.4000015258789, 83.19999694824219, 81.44999694824219, 83.6500015258789, 83.30000305175781, 83.55000305175781, 83.5999984741211, 83.4000015258789, 83.1500015258789, 84.1500015258789, 83.69999694824219, 84.05000305175781, 83.44999694824219, 83.5999984741211, 83.5999984741211, 83.55000305175781, 83.3499984741211, 83.75, 83.8499984741211, 83.69999694824219, 83.5999984741211, 83.80000305175781, 83.94999694824219, 84.0999984741211, 84.19999694824219, 83.75, 84.0, 83.3499984741211, 84.4000015258789, 84.5, 84.19999694824219, 84.8499984741211, 84.55000305175781, 84.3499984741211, 84.5999984741211, 83.4000015258789, 83.80000305175781, 84.5999984741211, 84.25, 84.4000015258789, 84.44999694824219, 84.8499984741211, 82.9000015258789, 84.80000305175781, 84.55000305175781, 84.3499984741211, 84.05000305175781, 84.3499984741211, 82.94999694824219, 84.6500015258789, 82.6500015258789, 84.3499984741211, 84.8499984741211, 85.1500015258789, 84.5999984741211, 84.55000305175781, 84.0999984741211, 84.6500015258789, 84.5999984741211, 84.4000015258789, 84.94999694824219, 84.8499984741211, 81.80000305175781, 84.30000305175781, 84.69999694824219, 83.25, 84.80000305175781, 84.5, 84.94999694824219, 83.9000015258789, 85.0999984741211, 84.75, 84.1500015258789, 84.8499984741211, 84.6500015258789, 83.19999694824219, 84.1500015258789, 83.55000305175781, 84.44999694824219, 84.5, 83.19999694824219, 84.30000305175781, 84.1500015258789, 84.4000015258789, 84.80000305175781, 84.3499984741211, 84.19999694824219, 83.19999694824219, 84.80000305175781, 84.5, 84.5999984741211, 82.4000015258789, 84.8499984741211, 85.0, 84.3499984741211, 85.0999984741211, 84.75, 84.75, 84.5, 84.5999984741211, 84.44999694824219, 84.9000015258789, 84.05000305175781, 84.94999694824219, 83.0999984741211, 82.4000015258789, 82.8499984741211, 84.1500015258789, 84.9000015258789, 84.9000015258789, 84.3499984741211, 85.1500015258789, 83.25, 85.19999694824219, 83.75, 84.55000305175781, 84.30000305175781, 84.1500015258789, 84.25, 82.80000305175781, 84.80000305175781, 84.25, 83.5999984741211, 83.44999694824219, 84.6500015258789, 85.30000305175781, 84.75, 85.1500015258789, 85.44999694824219, 84.4000015258789, 85.0999984741211, 84.25, 85.1500015258789, 84.94999694824219, 84.19999694824219, 83.6500015258789, 85.1500015258789, 85.1500015258789, 84.55000305175781, 84.44999694824219, 85.3499984741211, 84.3499984741211, 83.94999694824219, 85.3499984741211, 85.0999984741211, 85.1500015258789, 84.5, 84.6500015258789, 85.05000305175781, 85.19999694824219, 85.0, 85.0999984741211, 85.5999984741211, 85.30000305175781, 85.25, 85.75, 85.69999694824219, 85.4000015258789, 85.4000015258789, 85.0, 85.0, 85.19999694824219, 85.25, 85.05000305175781, 85.44999694824219, 85.1500015258789, 85.55000305175781, 85.05000305175781, 85.19999694824219, 85.5999984741211, 85.5, 84.8499984741211, 85.80000305175781, 85.19999694824219, 85.30000305175781, 85.1500015258789, 85.44999694824219, 85.30000305175781, 85.0, 85.19999694824219, 85.30000305175781, 85.25, 84.30000305175781, 84.8499984741211, 85.1500015258789, 85.5, 85.05000305175781, 85.1500015258789, 85.3499984741211, 85.25, 85.3499984741211, 85.30000305175781, 85.25, 85.30000305175781, 85.4000015258789, 84.44999694824219, 85.75, 85.80000305175781] 
Cluster_1_accuracy_50=[50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.900001525878906, 51.29999923706055, 56.150001525878906, 50.20000076293945, 50.25, 50.70000076293945, 50.150001525878906, 50.0, 50.95000076293945, 50.0, 49.900001525878906, 50.54999923706055, 50.599998474121094, 51.349998474121094, 50.900001525878906, 51.25, 51.20000076293945, 50.099998474121094, 50.04999923706055, 45.849998474121094, 46.75, 44.849998474121094, 48.54999923706055, 49.54999923706055, 50.0, 49.400001525878906, 50.25, 50.20000076293945, 50.0, 50.0, 50.0, 50.0, 50.0, 50.04999923706055, 55.29999923706055, 57.79999923706055, 50.0, 50.900001525878906, 51.75, 52.75, 57.75, 56.650001525878906, 58.150001525878906, 50.150001525878906, 51.70000076293945, 59.45000076293945, 60.0, 58.099998474121094, 59.900001525878906, 60.25, 59.45000076293945, 60.54999923706055, 57.150001525878906, 59.70000076293945, 60.5, 59.79999923706055, 58.599998474121094, 59.849998474121094, 59.79999923706055, 57.20000076293945, 54.45000076293945, 58.79999923706055, 61.349998474121094, 54.849998474121094, 58.650001525878906, 62.04999923706055, 62.849998474121094, 62.349998474121094, 59.5, 58.900001525878906, 63.20000076293945, 58.599998474121094, 62.75, 63.70000076293945, 64.3499984741211, 63.29999923706055, 61.45000076293945, 63.400001525878906, 57.900001525878906, 58.599998474121094, 62.900001525878906, 60.95000076293945, 64.0, 65.0999984741211, 63.0, 64.0999984741211, 61.75, 63.70000076293945, 61.5, 57.900001525878906, 57.150001525878906, 62.349998474121094, 61.099998474121094, 63.849998474121094, 64.1500015258789, 64.1500015258789, 64.5999984741211, 64.25, 64.8499984741211, 63.79999923706055, 64.80000305175781, 65.44999694824219, 64.5999984741211, 64.80000305175781, 65.05000305175781, 62.150001525878906, 63.599998474121094, 62.400001525878906, 63.900001525878906, 62.29999923706055, 63.25, 64.55000305175781, 65.8499984741211, 64.8499984741211, 61.150001525878906, 61.20000076293945, 65.69999694824219, 64.69999694824219, 65.5999984741211, 66.25, 63.95000076293945, 65.19999694824219, 64.94999694824219, 65.5, 65.75, 63.79999923706055, 65.80000305175781, 65.55000305175781, 66.05000305175781, 65.6500015258789, 63.95000076293945, 65.94999694824219, 61.75, 64.75, 65.19999694824219, 65.19999694824219, 64.94999694824219, 66.0, 66.1500015258789, 66.0999984741211, 65.8499984741211, 65.30000305175781, 65.69999694824219, 65.9000015258789, 64.0999984741211, 66.1500015258789, 65.75, 62.29999923706055, 66.05000305175781, 66.25, 66.4000015258789, 66.5999984741211, 65.3499984741211, 65.0999984741211, 64.80000305175781, 64.94999694824219, 66.69999694824219, 65.0, 65.19999694824219, 66.6500015258789, 65.69999694824219, 66.69999694824219, 65.69999694824219, 66.25, 66.25, 65.0, 65.75, 65.55000305175781, 66.5, 65.44999694824219, 65.44999694824219, 65.6500015258789, 65.9000015258789, 66.3499984741211, 61.349998474121094, 66.44999694824219, 66.3499984741211, 66.3499984741211, 66.6500015258789, 65.5, 65.5999984741211, 65.5999984741211, 67.30000305175781, 67.0, 66.44999694824219, 66.5, 68.05000305175781, 67.6500015258789, 63.20000076293945, 65.55000305175781, 62.5, 66.3499984741211, 66.5999984741211, 66.30000305175781, 67.6500015258789, 68.4000015258789, 68.1500015258789, 68.0, 67.5999984741211, 68.1500015258789, 68.0, 68.44999694824219, 68.6500015258789, 68.75, 64.0999984741211, 69.0999984741211, 68.8499984741211, 69.3499984741211, 67.5, 67.3499984741211, 67.19999694824219, 69.5, 70.25, 70.6500015258789, 67.30000305175781, 70.69999694824219, 69.5999984741211, 70.8499984741211, 70.30000305175781, 69.94999694824219, 69.94999694824219, 69.55000305175781, 66.94999694824219, 70.0, 69.19999694824219, 65.55000305175781, 69.1500015258789, 70.25, 69.75, 69.8499984741211, 68.55000305175781, 68.4000015258789, 68.30000305175781, 69.4000015258789, 69.05000305175781, 70.44999694824219, 68.30000305175781, 70.6500015258789, 68.75, 71.0, 68.0, 64.3499984741211, 65.94999694824219, 68.80000305175781, 70.5, 69.9000015258789, 69.69999694824219, 70.5999984741211, 68.9000015258789, 70.5, 69.94999694824219, 69.55000305175781, 70.19999694824219, 69.8499984741211, 71.30000305175781, 69.0, 68.80000305175781, 69.5, 69.19999694824219, 70.30000305175781, 68.69999694824219, 69.9000015258789, 70.8499984741211, 70.80000305175781, 68.25, 71.5999984741211, 69.6500015258789, 66.5999984741211, 70.30000305175781, 69.94999694824219, 70.5, 70.19999694824219, 69.0, 68.6500015258789, 70.30000305175781, 70.80000305175781, 70.30000305175781, 69.25, 70.8499984741211, 70.44999694824219, 70.44999694824219, 70.44999694824219, 70.6500015258789, 71.44999694824219, 71.69999694824219, 71.1500015258789, 69.80000305175781, 66.4000015258789, 69.0, 70.55000305175781, 66.44999694824219, 69.4000015258789, 71.19999694824219, 71.25, 71.30000305175781, 71.3499984741211, 72.0999984741211, 71.0999984741211, 71.6500015258789, 72.30000305175781, 72.0, 71.5, 71.5999984741211, 72.1500015258789, 72.0, 72.1500015258789, 71.6500015258789, 71.19999694824219, 69.19999694824219, 71.69999694824219, 70.94999694824219, 71.25, 71.94999694824219, 71.44999694824219, 71.19999694824219, 71.55000305175781, 70.30000305175781, 71.5, 64.05000305175781, 71.1500015258789, 71.30000305175781, 71.05000305175781, 71.8499984741211, 72.0999984741211, 72.05000305175781, 68.1500015258789, 70.69999694824219, 71.69999694824219, 71.6500015258789, 72.44999694824219, 71.25, 71.5999984741211, 71.0999984741211, 71.8499984741211, 70.75, 71.5999984741211, 71.5, 70.6500015258789, 71.05000305175781, 71.80000305175781, 71.6500015258789, 72.0999984741211, 72.05000305175781, 72.05000305175781, 71.80000305175781, 71.8499984741211, 71.94999694824219, 71.9000015258789, 71.19999694824219, 70.80000305175781, 63.400001525878906, 71.25, 71.55000305175781, 67.1500015258789, 71.05000305175781, 71.4000015258789, 72.19999694824219, 71.55000305175781, 69.44999694824219, 71.75, 72.5999984741211, 72.1500015258789, 71.8499984741211, 71.9000015258789, 71.55000305175781, 71.44999694824219, 71.94999694824219, 71.5999984741211, 72.30000305175781, 72.4000015258789, 72.44999694824219, 72.4000015258789, 72.9000015258789, 72.5999984741211, 72.0, 72.30000305175781, 72.55000305175781, 70.1500015258789, 72.3499984741211, 71.69999694824219, 72.1500015258789, 72.0, 72.05000305175781, 71.75, 71.8499984741211, 72.1500015258789, 71.8499984741211, 72.1500015258789, 71.19999694824219, 71.75, 72.0999984741211, 72.05000305175781, 71.1500015258789, 71.6500015258789, 65.3499984741211, 72.0, 68.30000305175781, 72.0, 70.19999694824219, 69.6500015258789, 70.6500015258789, 68.30000305175781, 70.44999694824219, 71.75, 71.0, 71.5, 71.30000305175781, 71.94999694824219, 71.94999694824219, 72.05000305175781, 71.4000015258789, 71.5999984741211, 70.1500015258789, 68.9000015258789, 69.80000305175781, 71.55000305175781, 71.94999694824219, 70.80000305175781, 67.05000305175781, 70.5, 71.80000305175781, 71.80000305175781, 71.9000015258789, 71.05000305175781, 72.19999694824219, 71.94999694824219, 71.5, 69.30000305175781, 72.25, 72.0999984741211, 71.94999694824219, 72.0, 71.25, 71.55000305175781, 72.3499984741211, 72.05000305175781, 71.9000015258789, 71.0999984741211, 71.4000015258789, 70.94999694824219, 71.94999694824219, 70.4000015258789, 72.5999984741211, 70.5, 71.8499984741211, 72.1500015258789, 72.05000305175781, 72.0999984741211, 71.69999694824219, 71.8499984741211, 71.5999984741211, 69.0999984741211, 71.5999984741211, 71.3499984741211, 71.6500015258789, 71.55000305175781, 71.6500015258789, 71.6500015258789, 71.69999694824219, 70.30000305175781, 71.44999694824219, 70.8499984741211, 71.1500015258789, 71.0, 71.0999984741211, 71.94999694824219, 72.25, 71.69999694824219, 71.3499984741211, 70.75, 71.19999694824219, 70.8499984741211, 71.55000305175781, 71.0, 69.44999694824219, 70.25, 71.5, 71.44999694824219, 71.75, 71.4000015258789, 71.4000015258789, 71.69999694824219, 69.8499984741211, 71.69999694824219, 71.8499984741211, 71.5999984741211, 71.44999694824219, 71.6500015258789, 70.5999984741211, 70.05000305175781, 70.94999694824219, 69.0999984741211, 71.19999694824219, 71.0999984741211] 
Cluster_2_accuracy_50=[2.5, 44.54999923706055, 49.849998474121094, 49.95000076293945, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.04999923706055, 50.04999923706055, 50.04999923706055, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 58.400001525878906, 53.099998474121094, 54.5, 55.25, 50.150001525878906, 53.79999923706055, 51.5, 53.54999923706055, 50.849998474121094, 50.400001525878906, 50.20000076293945, 62.54999923706055, 50.849998474121094, 63.650001525878906, 61.5, 50.150001525878906, 53.45000076293945, 52.75, 65.05000305175781, 50.04999923706055, 51.29999923706055, 62.95000076293945, 66.1500015258789, 65.0999984741211, 66.6500015258789, 64.3499984741211, 61.5, 61.099998474121094, 66.1500015258789, 65.0, 65.3499984741211, 65.4000015258789, 65.5999984741211, 64.05000305175781, 66.55000305175781, 62.75, 63.70000076293945, 63.099998474121094, 64.1500015258789, 60.400001525878906, 66.5999984741211, 66.5999984741211, 65.25, 66.4000015258789, 67.0999984741211, 64.0, 66.1500015258789, 67.0, 66.69999694824219, 67.0999984741211, 67.05000305175781, 67.55000305175781, 67.44999694824219, 66.9000015258789, 67.4000015258789, 67.5, 66.55000305175781, 67.55000305175781, 64.5999984741211, 67.5, 67.8499984741211, 68.0, 67.30000305175781, 67.5, 67.94999694824219, 67.5999984741211, 66.25, 67.44999694824219, 67.9000015258789, 67.1500015258789, 67.69999694824219, 68.1500015258789, 68.30000305175781, 67.5, 67.55000305175781, 67.44999694824219, 67.80000305175781, 68.1500015258789, 67.1500015258789, 68.25, 68.5, 68.3499984741211, 68.5999984741211, 68.44999694824219, 68.9000015258789, 68.55000305175781, 67.05000305175781, 69.3499984741211, 68.9000015258789, 66.05000305175781, 65.69999694824219, 66.8499984741211, 69.05000305175781, 66.69999694824219, 69.55000305175781, 70.1500015258789, 66.9000015258789, 69.0, 70.05000305175781, 68.3499984741211, 67.0999984741211, 70.0999984741211, 68.05000305175781, 70.19999694824219, 70.75, 70.6500015258789, 69.9000015258789, 70.30000305175781, 68.5, 70.30000305175781, 67.1500015258789, 68.8499984741211, 70.1500015258789, 70.30000305175781, 70.69999694824219, 70.55000305175781, 69.9000015258789, 70.80000305175781, 70.5, 70.5999984741211, 68.0999984741211, 71.0, 70.75, 71.05000305175781, 71.44999694824219, 70.19999694824219, 71.8499984741211, 72.0, 69.3499984741211, 69.69999694824219, 71.4000015258789, 71.8499984741211, 70.4000015258789, 71.75, 72.0999984741211, 72.0, 67.94999694824219, 72.69999694824219, 72.5, 70.80000305175781, 72.6500015258789, 72.4000015258789, 71.19999694824219, 72.19999694824219, 73.6500015258789, 73.19999694824219, 73.4000015258789, 73.05000305175781, 70.94999694824219, 73.0, 72.4000015258789, 73.25, 71.25, 73.44999694824219, 73.19999694824219, 73.3499984741211, 72.55000305175781, 72.69999694824219, 72.9000015258789, 72.30000305175781, 71.80000305175781, 73.5, 73.69999694824219, 72.8499984741211, 73.6500015258789, 74.0, 73.4000015258789, 73.8499984741211, 72.80000305175781, 73.6500015258789, 73.30000305175781, 73.5999984741211, 74.0, 74.05000305175781, 73.75, 74.25, 73.69999694824219, 73.5, 74.1500015258789, 74.69999694824219, 74.0999984741211, 74.3499984741211, 73.5999984741211, 73.44999694824219, 74.05000305175781, 74.1500015258789, 74.4000015258789, 74.19999694824219, 73.75, 74.4000015258789, 74.05000305175781, 74.0, 72.9000015258789, 72.94999694824219, 73.69999694824219, 73.94999694824219, 73.55000305175781, 74.25, 74.80000305175781, 74.25, 74.3499984741211, 74.55000305175781, 74.55000305175781, 75.5999984741211, 75.75, 74.94999694824219, 75.44999694824219, 75.75, 74.5999984741211, 75.19999694824219, 74.9000015258789, 75.80000305175781, 73.5, 74.5, 75.1500015258789, 74.55000305175781, 75.4000015258789, 75.30000305175781, 75.0, 74.9000015258789, 75.0, 75.4000015258789, 74.80000305175781, 74.80000305175781, 75.4000015258789, 75.69999694824219, 75.5, 75.4000015258789, 75.80000305175781, 75.30000305175781, 75.6500015258789, 75.25, 75.8499984741211, 75.69999694824219, 75.1500015258789, 75.44999694824219, 75.44999694824219, 75.0999984741211, 75.5, 76.0, 76.44999694824219, 75.05000305175781, 75.94999694824219, 75.44999694824219, 75.4000015258789, 75.9000015258789, 75.8499984741211, 76.25, 74.5999984741211, 75.6500015258789, 76.0999984741211, 75.69999694824219, 75.8499984741211, 75.0999984741211, 75.9000015258789, 75.19999694824219, 75.75, 76.4000015258789, 76.44999694824219, 75.80000305175781, 75.55000305175781, 76.0, 75.94999694824219, 75.5999984741211, 75.55000305175781, 75.6500015258789, 75.0, 74.5999984741211, 75.30000305175781, 74.5999984741211, 75.0, 76.5, 75.0999984741211, 75.3499984741211, 76.5, 75.3499984741211, 75.55000305175781, 74.94999694824219, 75.69999694824219, 74.5, 74.05000305175781, 75.5999984741211, 75.3499984741211, 75.80000305175781, 73.5, 76.0999984741211, 76.3499984741211, 75.3499984741211, 75.80000305175781, 75.05000305175781, 75.19999694824219, 75.30000305175781, 75.3499984741211, 74.8499984741211, 74.80000305175781, 74.75, 74.4000015258789, 74.6500015258789, 75.9000015258789, 76.5, 76.80000305175781, 76.30000305175781, 75.69999694824219, 76.0999984741211, 75.8499984741211, 76.1500015258789, 76.1500015258789, 75.8499984741211, 75.4000015258789, 76.0999984741211, 75.94999694824219, 75.8499984741211, 76.1500015258789, 75.80000305175781, 76.1500015258789, 76.1500015258789, 75.5999984741211, 75.55000305175781, 75.44999694824219, 75.9000015258789, 75.6500015258789, 76.1500015258789, 75.3499984741211, 75.3499984741211, 75.75, 74.80000305175781, 76.0, 75.5999984741211, 75.80000305175781, 75.6500015258789, 75.80000305175781, 76.05000305175781, 76.5999984741211, 76.55000305175781, 76.3499984741211, 75.75, 76.75, 76.0999984741211, 75.75, 76.0, 75.8499984741211, 75.55000305175781, 76.0999984741211, 76.05000305175781, 75.19999694824219, 76.3499984741211, 76.0999984741211, 75.4000015258789, 76.0999984741211, 75.5999984741211, 76.75, 76.30000305175781, 76.6500015258789, 76.3499984741211, 76.75, 76.25, 76.6500015258789, 76.6500015258789, 76.80000305175781, 76.4000015258789, 76.25, 75.94999694824219, 76.25, 77.0, 77.3499984741211, 76.5999984741211, 76.55000305175781, 76.1500015258789, 76.30000305175781, 76.5, 77.1500015258789, 76.44999694824219, 76.5, 76.25, 75.6500015258789, 75.75, 75.94999694824219, 76.1500015258789, 75.9000015258789, 76.1500015258789, 76.5, 76.05000305175781, 75.8499984741211, 75.44999694824219, 76.5999984741211, 76.44999694824219, 76.44999694824219, 75.80000305175781, 76.25, 75.5999984741211, 75.55000305175781, 76.05000305175781, 75.75, 75.8499984741211, 75.6500015258789, 75.3499984741211, 75.75, 74.69999694824219, 75.19999694824219, 75.5, 75.94999694824219, 75.05000305175781, 75.9000015258789, 75.55000305175781, 75.80000305175781, 76.0, 76.0999984741211, 75.94999694824219, 76.05000305175781, 76.19999694824219, 76.05000305175781, 76.0, 76.25, 76.05000305175781, 75.80000305175781, 75.94999694824219, 75.80000305175781, 75.75, 75.5, 76.1500015258789, 75.9000015258789, 75.6500015258789, 75.69999694824219, 76.4000015258789, 76.5, 76.05000305175781, 76.1500015258789, 76.05000305175781, 75.44999694824219, 76.19999694824219, 75.9000015258789, 75.94999694824219, 75.80000305175781, 75.9000015258789, 75.5999984741211, 75.55000305175781, 75.4000015258789, 75.69999694824219, 74.80000305175781, 76.25, 75.30000305175781, 75.05000305175781, 76.0, 75.5999984741211, 76.30000305175781, 76.4000015258789, 75.80000305175781, 76.0, 75.9000015258789, 75.80000305175781, 75.69999694824219, 75.8499984741211, 76.19999694824219, 76.1500015258789, 76.05000305175781, 75.75, 76.5999984741211, 75.6500015258789, 76.30000305175781, 76.3499984741211, 75.9000015258789, 76.25, 76.5, 77.19999694824219, 76.55000305175781, 76.69999694824219, 76.5999984741211, 76.4000015258789] 
Cluster_3_accuracy_50=[20.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 49.95000076293945, 56.95000076293945, 54.599998474121094, 66.30000305175781, 63.20000076293945, 67.75, 68.5, 65.5999984741211, 67.69999694824219, 68.19999694824219, 56.20000076293945, 56.150001525878906, 57.849998474121094, 68.3499984741211, 70.0999984741211, 66.05000305175781, 62.70000076293945, 58.599998474121094, 65.94999694824219, 57.900001525878906, 70.80000305175781, 70.8499984741211, 70.44999694824219, 70.25, 71.1500015258789, 71.4000015258789, 71.1500015258789, 70.4000015258789, 71.0999984741211, 71.75, 65.55000305175781, 72.44999694824219, 73.05000305175781, 72.8499984741211, 72.80000305175781, 72.3499984741211, 72.69999694824219, 73.4000015258789, 66.69999694824219, 73.4000015258789, 72.6500015258789, 73.44999694824219, 74.55000305175781, 74.9000015258789, 73.94999694824219, 72.1500015258789, 74.69999694824219, 75.55000305175781, 75.0999984741211, 73.94999694824219, 73.30000305175781, 75.55000305175781, 76.0999984741211, 77.0999984741211, 76.75, 76.5999984741211, 76.80000305175781, 77.25, 77.19999694824219, 77.0999984741211, 77.3499984741211, 77.9000015258789, 77.69999694824219, 77.94999694824219, 76.6500015258789, 78.44999694824219, 78.6500015258789, 78.05000305175781, 74.9000015258789, 78.5999984741211, 78.30000305175781, 78.8499984741211, 79.6500015258789, 79.6500015258789, 78.80000305175781, 79.5, 77.0, 78.05000305175781, 78.1500015258789, 79.19999694824219, 79.19999694824219, 79.19999694824219, 79.3499984741211, 78.55000305175781, 79.0999984741211, 79.55000305175781, 79.6500015258789, 79.5, 79.5999984741211, 79.9000015258789, 80.0, 80.0, 79.4000015258789, 76.69999694824219, 79.3499984741211, 80.1500015258789, 80.30000305175781, 81.0999984741211, 80.80000305175781, 80.94999694824219, 80.6500015258789, 80.44999694824219, 81.0999984741211, 80.94999694824219, 80.80000305175781, 78.25, 80.80000305175781, 81.0999984741211, 80.5, 80.80000305175781, 80.8499984741211, 81.1500015258789, 81.0, 81.5, 81.44999694824219, 80.9000015258789, 81.25, 81.4000015258789, 81.25, 81.1500015258789, 81.5999984741211, 80.55000305175781, 81.8499984741211, 81.4000015258789, 82.25, 81.05000305175781, 81.69999694824219, 82.0999984741211, 81.5, 81.25, 81.05000305175781, 81.9000015258789, 81.4000015258789, 82.0, 81.94999694824219, 81.30000305175781, 81.6500015258789, 81.44999694824219, 81.80000305175781, 81.9000015258789, 81.69999694824219, 81.1500015258789, 81.6500015258789, 81.8499984741211, 81.94999694824219, 81.44999694824219, 80.8499984741211, 81.0, 82.0999984741211, 81.44999694824219, 81.8499984741211, 81.94999694824219, 79.80000305175781, 82.0, 81.1500015258789, 81.94999694824219, 82.55000305175781, 80.94999694824219, 82.19999694824219, 82.5999984741211, 81.44999694824219, 82.5999984741211, 82.19999694824219, 82.44999694824219, 81.9000015258789, 82.1500015258789, 80.05000305175781, 82.0, 81.5, 82.3499984741211, 81.5999984741211, 81.80000305175781, 81.4000015258789, 81.44999694824219, 82.19999694824219, 81.9000015258789, 79.19999694824219, 82.3499984741211, 81.4000015258789, 82.44999694824219, 82.80000305175781, 82.8499984741211, 80.5999984741211, 82.8499984741211, 81.94999694824219, 82.9000015258789, 83.25, 82.4000015258789, 81.5999984741211, 82.8499984741211, 83.44999694824219, 82.5999984741211, 82.5999984741211, 83.25, 82.8499984741211, 83.1500015258789, 81.05000305175781, 83.0, 83.44999694824219, 81.94999694824219, 83.25, 83.19999694824219, 82.0999984741211, 82.4000015258789, 83.19999694824219, 83.25, 83.5999984741211, 80.94999694824219, 83.44999694824219, 83.05000305175781, 83.8499984741211, 81.9000015258789, 83.69999694824219, 83.0999984741211, 82.8499984741211, 82.44999694824219, 82.94999694824219, 83.9000015258789, 83.55000305175781, 83.30000305175781, 83.6500015258789, 83.5999984741211, 83.69999694824219, 83.44999694824219, 82.94999694824219, 82.80000305175781, 83.05000305175781, 83.5, 81.1500015258789, 82.80000305175781, 83.0999984741211, 82.9000015258789, 83.8499984741211, 82.9000015258789, 82.5999984741211, 82.94999694824219, 83.9000015258789, 84.0, 84.1500015258789, 82.0999984741211, 83.19999694824219, 83.5, 84.0999984741211, 82.94999694824219, 83.69999694824219, 84.05000305175781, 83.44999694824219, 83.8499984741211, 84.05000305175781, 83.75, 83.25, 82.5999984741211, 83.55000305175781, 83.4000015258789, 82.1500015258789, 83.75, 83.0999984741211, 83.69999694824219, 84.1500015258789, 82.0999984741211, 82.0999984741211, 83.55000305175781, 83.94999694824219, 82.80000305175781, 83.44999694824219, 83.94999694824219, 83.4000015258789, 84.1500015258789, 84.0, 83.5999984741211, 82.80000305175781, 84.44999694824219, 84.30000305175781, 83.75, 83.30000305175781, 83.19999694824219, 83.3499984741211, 84.05000305175781, 82.5999984741211, 83.94999694824219, 82.55000305175781, 83.94999694824219, 84.3499984741211, 84.0, 84.0, 84.3499984741211, 83.05000305175781, 83.94999694824219, 84.4000015258789, 83.19999694824219, 84.1500015258789, 83.5, 83.94999694824219, 84.19999694824219, 80.94999694824219, 84.19999694824219, 84.05000305175781, 83.5, 84.19999694824219, 83.69999694824219, 83.8499984741211, 84.05000305175781, 84.0, 84.4000015258789, 84.4000015258789, 84.5, 84.30000305175781, 84.5999984741211, 84.5, 82.75, 84.44999694824219, 84.25, 83.9000015258789, 84.30000305175781, 82.05000305175781, 79.30000305175781, 84.3499984741211, 84.0999984741211, 84.25, 84.30000305175781, 83.75, 84.30000305175781, 83.5999984741211, 83.75, 84.25, 84.3499984741211, 84.19999694824219, 84.05000305175781, 84.0999984741211, 82.0999984741211, 84.5999984741211, 84.0999984741211, 84.1500015258789, 84.94999694824219, 84.8499984741211, 84.5999984741211, 84.55000305175781, 84.9000015258789, 84.94999694824219, 85.0999984741211, 85.30000305175781, 85.1500015258789, 84.6500015258789, 84.69999694824219, 84.3499984741211, 84.8499984741211, 85.05000305175781, 85.0, 85.05000305175781, 84.75, 85.1500015258789, 85.05000305175781, 84.5999984741211, 85.05000305175781, 84.0999984741211, 84.55000305175781, 84.6500015258789, 84.8499984741211, 84.5999984741211, 84.6500015258789, 84.44999694824219, 84.5, 84.4000015258789, 84.6500015258789, 84.8499984741211, 84.80000305175781, 85.0999984741211, 85.25, 84.69999694824219, 84.3499984741211, 84.9000015258789, 85.25, 84.69999694824219, 85.25, 84.5999984741211, 85.1500015258789, 85.25, 84.94999694824219, 84.9000015258789, 85.19999694824219, 85.0, 85.25, 85.0999984741211, 85.0, 85.1500015258789, 85.05000305175781, 85.1500015258789, 84.80000305175781, 83.69999694824219, 85.44999694824219, 85.75, 85.05000305175781, 84.30000305175781, 84.9000015258789, 85.44999694824219, 85.6500015258789, 84.30000305175781, 85.0999984741211, 85.05000305175781, 84.30000305175781, 85.4000015258789, 85.4000015258789, 85.4000015258789, 85.5999984741211, 85.8499984741211, 83.6500015258789, 82.0999984741211, 85.9000015258789, 84.69999694824219, 85.4000015258789, 83.44999694824219, 84.75, 85.19999694824219, 85.6500015258789, 84.9000015258789, 85.30000305175781, 85.94999694824219, 85.55000305175781, 85.9000015258789, 84.4000015258789, 85.55000305175781, 85.8499984741211, 85.6500015258789, 85.80000305175781, 85.94999694824219, 84.9000015258789, 85.25, 85.25, 85.80000305175781, 85.55000305175781, 86.0999984741211, 85.9000015258789, 85.94999694824219, 86.30000305175781, 85.69999694824219, 85.5999984741211, 85.94999694824219, 85.19999694824219, 85.9000015258789, 85.19999694824219, 85.5999984741211, 85.94999694824219, 84.9000015258789, 86.25, 86.0999984741211, 86.4000015258789, 86.30000305175781, 85.80000305175781, 85.1500015258789, 86.0, 85.9000015258789, 86.05000305175781, 86.19999694824219, 86.30000305175781, 86.4000015258789, 86.55000305175781, 86.4000015258789, 86.25, 85.94999694824219, 86.0999984741211]
Cluster_4_accuracy_50=[50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.150001525878906, 50.25, 50.20000076293945, 50.0, 50.0, 50.0, 50.0, 50.04999923706055, 50.099998474121094, 53.04999923706055, 52.599998474121094, 52.849998474121094, 50.099998474121094, 54.04999923706055, 53.849998474121094, 53.349998474121094, 52.04999923706055, 53.25, 57.29999923706055, 53.54999923706055, 65.75, 61.650001525878906, 66.5, 64.5, 63.95000076293945, 64.25, 65.6500015258789, 61.400001525878906, 62.150001525878906, 68.69999694824219, 68.94999694824219, 66.1500015258789, 68.94999694824219, 66.19999694824219, 68.3499984741211, 64.94999694824219, 67.25, 70.05000305175781, 69.44999694824219, 66.25, 69.05000305175781, 70.0999984741211, 66.4000015258789, 67.44999694824219, 68.75, 70.3499984741211, 66.44999694824219, 70.1500015258789, 70.5, 71.30000305175781, 71.25, 67.44999694824219, 69.0, 67.69999694824219, 69.55000305175781, 69.75, 71.75, 70.3499984741211, 71.05000305175781, 68.55000305175781, 72.0999984741211, 71.6500015258789, 71.69999694824219, 71.9000015258789, 71.1500015258789, 70.8499984741211, 71.69999694824219, 71.9000015258789, 68.6500015258789, 71.55000305175781, 67.94999694824219, 72.69999694824219, 70.94999694824219, 72.44999694824219, 67.9000015258789, 70.0999984741211, 72.6500015258789, 72.5999984741211, 72.9000015258789, 71.6500015258789, 70.9000015258789, 73.0999984741211, 73.30000305175781, 71.25, 71.19999694824219, 72.25, 71.25, 68.25, 68.8499984741211, 73.1500015258789, 73.19999694824219, 69.8499984741211, 72.69999694824219, 72.5999984741211, 72.0, 72.69999694824219, 72.25, 73.0, 73.6500015258789, 72.8499984741211, 72.1500015258789, 72.8499984741211, 73.19999694824219, 72.80000305175781, 73.5, 74.0, 74.25, 70.4000015258789, 72.30000305175781, 74.25, 74.19999694824219, 73.94999694824219, 72.94999694824219, 74.1500015258789, 72.9000015258789, 74.44999694824219, 74.5, 74.4000015258789, 73.55000305175781, 73.4000015258789, 74.8499984741211, 75.0, 74.75, 73.19999694824219, 75.1500015258789, 74.94999694824219, 74.69999694824219, 74.5999984741211, 75.1500015258789, 75.44999694824219, 74.55000305175781, 75.4000015258789, 75.44999694824219, 75.19999694824219, 75.5, 74.0, 75.0999984741211, 75.25, 74.9000015258789, 75.3499984741211, 75.44999694824219, 76.4000015258789, 73.80000305175781, 75.9000015258789, 75.94999694824219, 75.80000305175781, 76.05000305175781, 75.75, 76.19999694824219, 75.69999694824219, 75.3499984741211, 75.80000305175781, 75.25, 75.5999984741211, 73.0, 75.9000015258789, 76.30000305175781, 76.3499984741211, 75.3499984741211, 76.44999694824219, 76.19999694824219, 76.0999984741211, 76.5, 76.30000305175781, 76.0, 76.25, 76.0999984741211, 76.44999694824219, 76.19999694824219, 75.94999694824219, 76.44999694824219, 77.05000305175781, 76.6500015258789, 76.6500015258789, 77.0999984741211, 77.05000305175781, 76.94999694824219, 76.94999694824219, 77.1500015258789, 76.80000305175781, 77.6500015258789, 76.94999694824219, 77.55000305175781, 77.9000015258789, 77.30000305175781, 78.1500015258789, 77.3499984741211, 77.75, 77.5, 77.4000015258789, 76.8499984741211, 77.30000305175781, 77.05000305175781, 77.0, 77.44999694824219, 77.19999694824219, 77.9000015258789, 77.25, 77.75, 78.0, 77.75, 77.75, 77.75, 77.5, 78.3499984741211, 77.6500015258789, 77.6500015258789, 78.4000015258789, 78.5, 77.9000015258789, 77.94999694824219, 77.05000305175781, 79.3499984741211, 79.0, 78.55000305175781, 78.5999984741211, 77.19999694824219, 78.55000305175781, 78.9000015258789, 79.1500015258789, 78.8499984741211, 79.0999984741211, 79.0, 79.3499984741211, 77.8499984741211, 79.3499984741211, 78.4000015258789, 79.30000305175781, 79.0999984741211, 79.55000305175781, 79.55000305175781, 79.3499984741211, 79.0, 79.4000015258789, 79.55000305175781, 79.80000305175781, 79.55000305175781, 80.05000305175781, 79.9000015258789, 80.05000305175781, 79.69999694824219, 79.94999694824219, 79.6500015258789, 79.9000015258789, 79.30000305175781, 79.3499984741211, 79.94999694824219, 79.8499984741211, 78.8499984741211, 79.94999694824219, 79.5, 79.8499984741211, 79.5999984741211, 79.55000305175781, 79.5, 79.6500015258789, 79.5, 79.55000305175781, 79.8499984741211, 79.5999984741211, 80.1500015258789, 79.94999694824219, 79.8499984741211, 79.80000305175781, 80.19999694824219, 80.44999694824219, 79.9000015258789, 80.5999984741211, 80.05000305175781, 80.6500015258789, 80.6500015258789, 80.6500015258789, 79.75, 79.4000015258789, 80.44999694824219, 80.3499984741211, 80.30000305175781, 80.9000015258789, 80.6500015258789, 80.55000305175781, 80.3499984741211, 81.0, 81.05000305175781, 80.69999694824219, 79.9000015258789, 80.8499984741211, 80.5999984741211, 81.5, 80.44999694824219, 81.19999694824219, 81.0999984741211, 81.05000305175781, 81.19999694824219, 81.4000015258789, 81.30000305175781, 80.9000015258789, 81.5999984741211, 81.44999694824219, 81.30000305175781, 82.05000305175781, 81.8499984741211, 81.75, 82.0, 81.69999694824219, 81.69999694824219, 80.9000015258789, 81.94999694824219, 81.69999694824219, 81.44999694824219, 81.5999984741211, 80.80000305175781, 81.25, 81.75, 81.5999984741211, 80.80000305175781, 82.0, 81.3499984741211, 81.30000305175781, 81.69999694824219, 82.05000305175781, 81.05000305175781, 77.6500015258789, 81.94999694824219, 81.55000305175781, 81.94999694824219, 82.0999984741211, 81.80000305175781, 81.75, 81.8499984741211, 81.8499984741211, 82.0, 81.6500015258789, 82.0999984741211, 82.0, 81.3499984741211, 81.44999694824219, 81.6500015258789, 82.30000305175781, 81.6500015258789, 82.19999694824219, 81.94999694824219, 80.69999694824219, 82.3499984741211, 82.0999984741211, 82.0, 81.5999984741211, 82.0999984741211, 82.3499984741211, 81.9000015258789, 82.5, 82.05000305175781, 82.5999984741211, 82.5, 82.0999984741211, 82.1500015258789, 82.44999694824219, 82.69999694824219, 82.3499984741211, 81.75, 83.0999984741211, 82.9000015258789, 82.6500015258789, 82.0999984741211, 82.3499984741211, 82.5999984741211, 82.44999694824219, 82.80000305175781, 82.80000305175781, 82.5999984741211, 82.6500015258789, 83.0999984741211, 82.75, 81.69999694824219, 83.0, 82.19999694824219, 82.55000305175781, 82.8499984741211, 80.5999984741211, 83.05000305175781, 83.05000305175781, 82.6500015258789, 82.30000305175781, 82.8499984741211, 82.55000305175781, 82.5, 82.75, 82.80000305175781, 82.75, 82.75, 82.8499984741211, 83.1500015258789, 82.6500015258789, 82.94999694824219, 83.0999984741211, 83.19999694824219, 83.44999694824219, 82.6500015258789, 82.25, 83.19999694824219, 83.05000305175781, 83.30000305175781, 83.3499984741211, 83.19999694824219, 83.4000015258789, 83.55000305175781, 83.80000305175781, 83.6500015258789, 83.44999694824219, 83.69999694824219, 83.80000305175781, 83.80000305175781, 84.0, 83.69999694824219, 83.6500015258789, 83.8499984741211, 83.5999984741211, 83.69999694824219, 83.30000305175781, 83.4000015258789, 83.44999694824219, 83.30000305175781, 83.5999984741211, 83.44999694824219, 83.44999694824219, 83.8499984741211, 83.75, 83.5, 83.30000305175781, 83.5999984741211, 83.30000305175781, 83.30000305175781, 83.6500015258789, 83.69999694824219, 83.3499984741211, 83.6500015258789, 83.9000015258789, 83.55000305175781, 83.6500015258789, 83.69999694824219, 84.0999984741211, 84.19999694824219, 83.75, 83.69999694824219, 84.0, 83.69999694824219, 83.55000305175781, 83.75, 83.5999984741211, 83.6500015258789, 83.69999694824219, 83.69999694824219, 83.5999984741211, 83.05000305175781, 83.30000305175781, 83.69999694824219, 83.6500015258789, 83.5, 83.55000305175781, 83.6500015258789, 83.5999984741211, 83.5999984741211, 83.9000015258789, 83.0, 83.69999694824219, 83.19999694824219, 83.3499984741211, 84.0, 84.0, 83.8499984741211, 83.8499984741211, 83.9000015258789, 83.9000015258789, 83.6500015258789, 83.75, 83.69999694824219, 83.8499984741211]

#50 antenna lr=0.001
Cluster_0_accuracy_50_small=[49.75, 50.0, 46.95000076293945, 46.150001525878906, 3.0, 42.810001373291016, 40.38999876976013, 50.0, 55.150001525878906, 60.45000076293945, 66.19999694824219, 69.55000305175781, 55.45000076293945, 65.5, 69.19999694824219, 72.1500015258789, 64.4000015258789, 63.849998474121094, 62.099998474121094, 66.6500015258789, 66.3499984741211, 39.849998474121094, 60.849998474121094, 68.05000305175781, 59.150001525878906, 68.30000305175781, 67.1500015258789, 68.3499984741211, 68.8499984741211, 58.45000076293945, 71.8499984741211, 66.94999694824219, 68.4000015258789, 61.0, 64.44999694824219, 65.3499984741211, 62.0, 61.04999923706055, 63.70000076293945, 66.75, 59.150001525878906, 58.54999923706055, 65.69999694824219, 66.69999694824219, 63.400001525878906, 61.400001525878906, 62.20000076293945, 54.70000076293945, 64.30000305175781, 66.05000305175781, 62.349998474121094, 67.05000305175781, 65.1500015258789, 66.44999694824219, 66.8499984741211, 70.4000015258789, 69.1500015258789, 70.44999694824219, 62.5, 68.3499984741211, 66.4000015258789, 63.45000076293945, 64.80000305175781, 63.04999923706055, 67.75, 69.0, 67.75, 65.94999694824219, 66.5, 61.150001525878906, 62.20000076293945, 66.0, 66.25, 66.1500015258789, 65.55000305175781, 66.19999694824219, 65.3499984741211, 67.30000305175781, 69.25, 71.19999694824219, 71.44999694824219, 68.80000305175781, 68.30000305175781, 67.1500015258789, 67.3499984741211, 63.54999923706055, 63.150001525878906, 63.20000076293945, 67.4000015258789, 61.25, 56.79999923706055, 63.20000076293945, 63.45000076293945, 64.80000305175781, 66.9000015258789, 68.30000305175781, 63.349998474121094, 70.5, 68.5, 70.0, 70.5, 69.4000015258789, 69.0999984741211, 69.19999694824219, 68.19999694824219, 68.69999694824219, 71.44999694824219, 72.19999694824219, 72.69999694824219, 72.05000305175781, 71.75, 71.75, 69.9000015258789, 70.1500015258789, 72.9000015258789, 71.6500015258789, 69.8499984741211, 72.5999984741211, 66.0, 74.05000305175781, 72.3499984741211, 75.05000305175781, 69.8499984741211, 71.25, 71.5, 71.80000305175781, 68.5999984741211, 72.3499984741211, 72.9000015258789, 70.75, 66.94999694824219, 71.30000305175781, 71.9000015258789, 71.0999984741211, 69.44999694824219, 69.25, 68.5999984741211, 65.69999694824219, 65.5, 66.9000015258789, 53.349998474121094, 57.150001525878906, 64.5999984741211, 65.0, 67.44999694824219, 66.55000305175781, 70.5, 65.0, 68.1500015258789, 70.55000305175781, 69.44999694824219, 70.75, 66.69999694824219, 66.30000305175781, 68.30000305175781, 70.1500015258789, 62.650001525878906, 69.25, 67.30000305175781, 68.6500015258789, 64.8499984741211, 67.1500015258789, 70.5, 70.44999694824219, 68.94999694824219, 67.80000305175781, 68.44999694824219, 66.5, 71.6500015258789, 71.30000305175781, 71.25, 72.19999694824219, 74.3499984741211, 67.69999694824219, 71.19999694824219, 69.44999694824219, 68.30000305175781, 70.75, 64.5999984741211, 68.5999984741211, 63.25, 66.94999694824219, 71.25, 73.0, 71.94999694824219, 71.55000305175781, 73.9000015258789, 73.3499984741211, 71.4000015258789, 73.4000015258789, 73.8499984741211, 72.0, 69.8499984741211, 71.44999694824219, 61.95000076293945, 67.25, 67.05000305175781, 70.30000305175781, 66.4000015258789, 69.4000015258789, 69.94999694824219, 68.55000305175781, 71.1500015258789, 72.75, 62.20000076293945, 65.0999984741211, 72.69999694824219, 70.3499984741211, 72.30000305175781, 72.44999694824219, 72.05000305175781, 71.3499984741211, 72.3499984741211, 72.4000015258789, 72.8499984741211, 70.44999694824219, 69.19999694824219, 70.9000015258789, 70.05000305175781, 67.5999984741211, 70.55000305175781, 67.69999694824219, 70.05000305175781, 72.69999694824219, 71.30000305175781, 69.1500015258789, 70.69999694824219, 72.25, 72.8499984741211, 74.80000305175781, 72.9000015258789, 69.5, 66.69999694824219, 71.6500015258789, 71.8499984741211, 68.8499984741211, 71.0999984741211, 68.0999984741211, 68.94999694824219, 71.05000305175781, 71.25, 69.80000305175781, 70.44999694824219, 72.25, 71.4000015258789, 70.6500015258789, 69.5, 70.3499984741211, 70.0, 71.69999694824219, 69.94999694824219, 70.44999694824219, 69.80000305175781, 70.8499984741211, 71.5999984741211, 67.1500015258789, 71.4000015258789, 70.4000015258789, 70.75, 71.25, 68.30000305175781, 68.80000305175781, 65.44999694824219, 69.8499984741211, 69.80000305175781, 67.69999694824219, 68.6500015258789, 72.0999984741211, 72.8499984741211, 71.8499984741211, 71.75, 71.5, 71.05000305175781, 70.30000305175781, 70.05000305175781, 71.19999694824219, 67.75, 71.44999694824219, 69.1500015258789, 69.69999694824219, 69.5999984741211, 66.6500015258789, 66.94999694824219, 68.5999984741211, 68.3499984741211, 67.1500015258789, 68.44999694824219, 70.4000015258789, 66.19999694824219, 68.5, 70.8499984741211, 69.30000305175781, 69.0, 68.0999984741211, 66.94999694824219, 69.94999694824219, 68.0, 66.5, 67.19999694824219, 65.44999694824219, 65.25, 61.54999923706055, 60.95000076293945, 63.0, 67.75, 66.1500015258789, 64.55000305175781, 63.650001525878906, 64.25, 66.30000305175781, 67.19999694824219, 67.44999694824219, 63.400001525878906, 68.25, 68.3499984741211, 67.25, 67.0, 67.25, 69.5999984741211, 67.6500015258789, 67.94999694824219, 67.9000015258789, 65.05000305175781, 65.4000015258789, 64.80000305175781, 67.19999694824219, 67.44999694824219, 67.4000015258789, 66.55000305175781, 66.19999694824219, 68.80000305175781, 70.75, 70.69999694824219, 70.8499984741211, 63.75, 68.5999984741211, 65.5999984741211, 68.19999694824219, 65.0, 64.69999694824219, 65.8499984741211, 67.8499984741211, 67.0999984741211, 65.69999694824219, 64.0999984741211, 64.1500015258789, 65.25, 66.44999694824219, 66.5999984741211, 66.30000305175781, 67.55000305175781, 67.80000305175781, 66.0, 67.05000305175781, 67.5999984741211, 68.0999984741211, 64.05000305175781, 65.5, 67.25, 64.94999694824219, 62.70000076293945, 62.150001525878906, 62.79999923706055, 60.29999923706055, 60.45000076293945, 62.0, 62.95000076293945, 60.849998474121094, 62.150001525878906, 62.849998474121094, 62.099998474121094, 62.95000076293945, 65.30000305175781, 64.44999694824219, 63.25, 63.349998474121094, 63.20000076293945, 63.54999923706055, 64.8499984741211, 65.30000305175781, 68.0, 67.30000305175781, 67.80000305175781, 68.55000305175781, 70.5999984741211, 68.5999984741211, 70.69999694824219, 72.55000305175781, 72.4000015258789, 71.1500015258789, 70.75, 70.4000015258789, 67.4000015258789, 72.55000305175781, 72.0999984741211, 72.30000305175781, 71.6500015258789, 72.1500015258789, 69.55000305175781, 68.30000305175781, 67.19999694824219, 67.80000305175781, 68.55000305175781, 70.30000305175781, 69.44999694824219, 70.55000305175781, 69.44999694824219, 68.3499984741211, 67.80000305175781, 68.8499984741211, 67.75, 67.4000015258789, 66.8499984741211, 68.0, 68.05000305175781, 69.0999984741211, 68.44999694824219, 69.94999694824219, 69.69999694824219, 68.75, 69.4000015258789, 69.75, 67.0, 69.0, 67.8499984741211, 65.75, 69.0, 67.75, 64.69999694824219, 64.8499984741211, 65.25, 65.3499984741211, 64.9000015258789, 65.9000015258789, 67.44999694824219, 67.0, 68.69999694824219, 67.8499984741211, 68.5, 69.1500015258789, 69.19999694824219, 69.4000015258789, 68.0999984741211, 67.75, 67.75, 67.9000015258789, 67.44999694824219, 67.9000015258789, 65.44999694824219, 67.0, 67.94999694824219, 68.55000305175781, 70.6500015258789, 71.4000015258789, 71.5, 70.0, 69.3499984741211, 68.69999694824219, 67.8499984741211, 67.0, 68.30000305175781, 67.80000305175781, 67.94999694824219, 68.8499984741211, 67.55000305175781, 68.8499984741211, 67.0999984741211, 65.5, 62.79999923706055, 66.25, 66.69999694824219, 70.75, 70.55000305175781, 68.6500015258789, 71.5, 72.19999694824219, 70.4000015258789, 67.69999694824219, 68.4000015258789, 71.3499984741211, 67.1500015258789, 67.4000015258789, 69.30000305175781, 69.05000305175781, 67.8499984741211, 63.400001525878906, 65.5999984741211, 67.4000015258789, 68.6500015258789, 68.9000015258789, 68.25, 68.55000305175781, 67.4000015258789, 66.80000305175781, 66.8499984741211, 66.0, 66.30000305175781, 67.0, 66.05000305175781, 66.0]
Cluster_1_accuracy_50_small=[49.25, 49.95000076293945, 50.04999923706055, 49.95000076293945, 54.29999923706055, 53.900001525878906, 52.75, 53.599998474121094, 50.150001525878906, 50.95000076293945, 55.099998474121094, 51.599998474121094, 52.599998474121094, 58.349998474121094, 59.25, 58.900001525878906, 53.650001525878906, 57.650001525878906, 54.150001525878906, 58.150001525878906, 51.54999923706055, 52.900001525878906, 54.5, 54.70000076293945, 52.29999923706055, 50.25, 54.04999923706055, 52.900001525878906, 53.349998474121094, 54.5, 55.75, 53.0, 51.70000076293945, 49.099998474121094, 52.0, 55.400001525878906, 54.0, 55.849998474121094, 55.70000076293945, 51.25, 51.400001525878906, 51.95000076293945, 47.099998474121094, 45.29999923706055, 48.400001525878906, 50.349998474121094, 51.349998474121094, 53.099998474121094, 53.45000076293945, 55.0, 55.650001525878906, 54.95000076293945, 57.45000076293945, 55.20000076293945, 56.54999923706055, 56.650001525878906, 56.79999923706055, 55.04999923706055, 54.04999923706055, 54.04999923706055, 55.20000076293945, 56.650001525878906, 56.79999923706055, 54.95000076293945, 53.349998474121094, 54.400001525878906, 53.849998474121094, 55.25, 56.849998474121094, 55.20000076293945, 53.20000076293945, 50.650001525878906, 49.650001525878906, 57.900001525878906, 59.0, 56.599998474121094, 55.25, 57.54999923706055, 56.54999923706055, 57.650001525878906, 58.45000076293945, 59.349998474121094, 56.599998474121094, 57.599998474121094, 54.900001525878906, 56.20000076293945, 56.45000076293945, 57.54999923706055, 57.20000076293945, 57.0, 56.349998474121094, 57.45000076293945, 55.849998474121094, 54.75, 54.599998474121094, 55.29999923706055, 55.349998474121094, 57.650001525878906, 57.150001525878906, 59.75, 58.79999923706055, 58.150001525878906, 59.95000076293945, 56.95000076293945, 56.95000076293945, 55.45000076293945, 55.0, 53.95000076293945, 54.75, 57.349998474121094, 53.04999923706055, 53.20000076293945, 52.95000076293945, 54.099998474121094, 54.400001525878906, 51.900001525878906, 52.54999923706055, 52.29999923706055, 54.599998474121094, 52.75, 53.04999923706055, 52.20000076293945, 49.29999923706055, 50.75, 53.29999923706055, 53.099998474121094, 51.599998474121094, 53.75, 53.650001525878906, 53.150001525878906, 50.900001525878906, 53.900001525878906, 50.04999923706055, 49.400001525878906, 49.54999923706055, 52.599998474121094, 49.75, 48.349998474121094, 43.0, 52.29999923706055, 51.79999923706055, 52.650001525878906, 55.099998474121094, 54.54999923706055, 53.349998474121094, 52.54999923706055, 50.349998474121094, 50.79999923706055, 50.400001525878906, 51.150001525878906, 50.75, 51.5, 51.20000076293945, 53.400001525878906, 51.650001525878906, 51.04999923706055, 51.0, 52.20000076293945, 53.95000076293945, 51.599998474121094, 55.54999923706055, 55.25, 53.849998474121094, 54.650001525878906, 53.70000076293945, 55.400001525878906, 53.349998474121094, 55.650001525878906, 56.099998474121094, 54.20000076293945, 55.45000076293945, 54.29999923706055, 55.04999923706055, 52.599998474121094, 53.900001525878906, 52.70000076293945, 51.400001525878906, 53.0, 53.20000076293945, 51.20000076293945, 55.29999923706055, 55.400001525878906, 54.70000076293945, 56.400001525878906, 55.5, 54.099998474121094, 54.150001525878906, 55.75, 55.54999923706055, 55.54999923706055, 57.099998474121094, 57.099998474121094, 56.70000076293945, 55.25, 55.04999923706055, 54.54999923706055, 54.5, 53.849998474121094, 52.54999923706055, 53.04999923706055, 56.349998474121094, 55.45000076293945, 53.849998474121094, 55.5, 55.5, 56.150001525878906, 56.29999923706055, 57.45000076293945, 58.45000076293945, 54.45000076293945, 56.650001525878906, 57.150001525878906, 56.400001525878906, 55.400001525878906, 56.25, 56.849998474121094, 55.650001525878906, 55.349998474121094, 54.95000076293945, 56.150001525878906, 54.95000076293945, 55.25, 55.20000076293945, 54.79999923706055, 54.20000076293945, 53.29999923706055, 53.54999923706055, 52.70000076293945, 53.5, 52.900001525878906, 53.599998474121094, 53.099998474121094, 53.599998474121094, 52.54999923706055, 52.04999923706055, 51.25, 52.95000076293945, 51.5, 52.20000076293945, 53.75, 52.0, 53.20000076293945, 52.599998474121094, 51.70000076293945, 52.79999923706055, 52.79999923706055, 53.099998474121094, 52.20000076293945, 52.349998474121094, 50.599998474121094, 50.099998474121094, 52.0, 51.79999923706055, 52.29999923706055, 54.20000076293945, 52.849998474121094, 52.400001525878906, 52.20000076293945, 51.79999923706055, 52.900001525878906, 53.79999923706055, 53.150001525878906, 52.70000076293945, 52.70000076293945, 52.900001525878906, 53.79999923706055, 53.29999923706055, 53.849998474121094, 54.0, 53.150001525878906, 53.54999923706055, 52.849998474121094, 53.900001525878906, 54.95000076293945, 54.400001525878906, 54.70000076293945, 53.650001525878906, 53.95000076293945, 54.5, 53.349998474121094, 53.0, 53.900001525878906, 53.349998474121094, 52.150001525878906, 52.349998474121094, 52.79999923706055, 53.099998474121094, 52.900001525878906, 51.5, 53.45000076293945, 54.150001525878906, 54.20000076293945, 54.900001525878906, 54.25, 54.900001525878906, 54.599998474121094, 54.25, 54.20000076293945, 54.54999923706055, 54.150001525878906, 54.54999923706055, 54.900001525878906, 53.79999923706055, 54.099998474121094, 54.04999923706055, 55.650001525878906, 54.20000076293945, 54.79999923706055, 55.150001525878906, 53.45000076293945, 53.70000076293945, 54.70000076293945, 53.29999923706055, 53.70000076293945, 54.54999923706055, 54.349998474121094, 53.900001525878906, 54.54999923706055, 55.29999923706055, 56.0, 54.400001525878906, 54.95000076293945, 54.900001525878906, 54.349998474121094, 54.54999923706055, 53.349998474121094, 53.400001525878906, 54.599998474121094, 54.54999923706055, 53.599998474121094, 54.70000076293945, 55.150001525878906, 53.04999923706055, 53.650001525878906, 55.900001525878906, 54.599998474121094, 53.70000076293945, 55.29999923706055, 55.150001525878906, 52.79999923706055, 53.5, 52.70000076293945, 53.900001525878906, 54.04999923706055, 54.150001525878906, 53.25, 54.5, 54.5, 54.25, 53.599998474121094, 53.20000076293945, 53.75, 50.95000076293945, 50.900001525878906, 50.400001525878906, 55.099998474121094, 54.95000076293945, 55.849998474121094, 54.900001525878906, 53.150001525878906, 53.95000076293945, 58.599998474121094, 57.29999923706055, 58.20000076293945, 57.5, 57.5, 56.45000076293945, 56.849998474121094, 56.150001525878906, 57.45000076293945, 58.04999923706055, 58.25, 60.0, 59.54999923706055, 60.150001525878906, 59.849998474121094, 60.849998474121094, 60.650001525878906, 61.5, 61.849998474121094, 60.849998474121094, 61.349998474121094, 60.599998474121094, 59.849998474121094, 58.79999923706055, 60.79999923706055, 60.099998474121094, 60.849998474121094, 59.04999923706055, 59.900001525878906, 59.54999923706055, 59.0, 57.45000076293945, 58.5, 58.04999923706055, 58.20000076293945, 58.400001525878906, 57.400001525878906, 59.599998474121094, 59.54999923706055, 57.04999923706055, 57.54999923706055, 57.650001525878906, 58.900001525878906, 56.849998474121094, 59.599998474121094, 59.5, 58.650001525878906, 58.650001525878906, 59.849998474121094, 59.25, 57.0, 56.5, 59.0, 59.5, 60.150001525878906, 60.849998474121094, 58.70000076293945, 59.900001525878906, 60.400001525878906, 58.45000076293945, 58.45000076293945, 57.20000076293945, 57.25, 56.900001525878906, 58.04999923706055, 56.54999923706055, 55.5, 56.25, 58.400001525878906, 57.349998474121094, 60.150001525878906, 58.70000076293945, 57.349998474121094, 57.54999923706055, 56.04999923706055, 58.5, 57.25, 56.70000076293945, 56.599998474121094, 57.70000076293945, 57.45000076293945, 56.900001525878906, 58.349998474121094, 58.650001525878906, 56.650001525878906, 56.20000076293945, 58.70000076293945, 60.75, 60.95000076293945, 59.349998474121094, 59.150001525878906, 59.20000076293945, 60.75, 59.599998474121094, 60.099998474121094, 59.79999923706055, 60.900001525878906, 61.25, 60.95000076293945, 59.5, 59.849998474121094, 60.45000076293945, 59.0, 58.45000076293945, 57.5, 58.150001525878906, 57.29999923706055, 58.650001525878906, 57.25, 60.29999923706055, 58.849998474121094, 58.599998474121094, 57.150001525878906, 57.20000076293945, 59.400001525878906, 58.349998474121094, 58.45000076293945, 58.25, 58.900001525878906, 57.20000076293945, 57.349998474121094, 56.900001525878906, 58.79999923706055, 57.04999923706055, 57.04999923706055, 56.0, 56.25, 56.25, 55.04999923706055, 55.849998474121094, 56.099998474121094, 54.349998474121094, 55.150001525878906, 56.099998474121094, 56.25, 55.04999923706055, 56.79999923706055, 56.29999923706055, 56.54999923706055]
Cluster_2_accuracy_50_small=[36.25, 53.650001525878906, 49.45000076293945, 53.25, 55.25, 53.5, 49.099998474121094, 52.150001525878906, 51.400001525878906, 53.79999923706055, 51.099998474121094, 55.150001525878906, 53.900001525878906, 55.70000076293945, 54.45000076293945, 55.29999923706055, 50.5, 56.70000076293945, 59.650001525878906, 50.099998474121094, 53.650001525878906, 56.849998474121094, 51.29999923706055, 55.25, 52.400001525878906, 55.29999923706055, 55.099998474121094, 57.150001525878906, 53.650001525878906, 56.29999923706055, 59.099998474121094, 55.650001525878906, 52.79999923706055, 54.0, 56.29999923706055, 54.45000076293945, 51.599998474121094, 53.5, 50.75, 46.54999923706055, 55.25, 57.20000076293945, 59.400001525878906, 53.54999923706055, 59.75, 56.45000076293945, 61.599998474121094, 53.0, 57.349998474121094, 55.75, 57.900001525878906, 58.04999923706055, 56.75, 58.599998474121094, 58.70000076293945, 52.04999923706055, 51.900001525878906, 53.04999923706055, 55.099998474121094, 58.5, 50.29999923706055, 53.79999923706055, 58.099998474121094, 60.20000076293945, 57.400001525878906, 51.349998474121094, 54.349998474121094, 50.70000076293945, 50.25, 54.5, 56.20000076293945, 55.95000076293945, 57.79999923706055, 56.849998474121094, 52.54999923706055, 51.54999923706055, 58.29999923706055, 50.900001525878906, 48.900001525878906, 49.25, 50.70000076293945, 56.20000076293945, 50.95000076293945, 52.849998474121094, 56.599998474121094, 56.0, 50.900001525878906, 56.150001525878906, 57.900001525878906, 58.25, 55.29999923706055, 54.25, 56.349998474121094, 53.5, 49.45000076293945, 51.599998474121094, 53.849998474121094, 53.04999923706055, 56.29999923706055, 53.29999923706055, 53.70000076293945, 55.349998474121094, 57.099998474121094, 56.099998474121094, 53.29999923706055, 51.349998474121094, 54.400001525878906, 53.95000076293945, 52.650001525878906, 54.650001525878906, 56.0, 55.04999923706055, 54.650001525878906, 55.45000076293945, 54.54999923706055, 57.650001525878906, 57.650001525878906, 56.599998474121094, 56.79999923706055, 57.0, 60.900001525878906, 59.70000076293945, 59.25, 56.5, 58.599998474121094, 58.45000076293945, 53.599998474121094, 58.650001525878906, 58.400001525878906, 52.29999923706055, 52.54999923706055, 52.79999923706055, 57.099998474121094, 59.29999923706055, 57.70000076293945, 58.150001525878906, 59.5, 58.900001525878906, 59.599998474121094, 59.099998474121094, 60.04999923706055, 59.099998474121094, 57.150001525878906, 57.25, 55.79999923706055, 54.70000076293945, 57.04999923706055, 58.5, 58.5, 59.95000076293945, 58.5, 59.599998474121094, 60.04999923706055, 59.900001525878906, 53.70000076293945, 56.400001525878906, 60.75, 59.20000076293945, 61.599998474121094, 60.75, 59.099998474121094, 59.349998474121094, 60.29999923706055, 59.5, 59.900001525878906, 59.25, 60.79999923706055, 60.79999923706055, 57.0, 58.54999923706055, 56.150001525878906, 60.849998474121094, 59.900001525878906, 61.400001525878906, 61.099998474121094, 58.349998474121094, 60.95000076293945, 54.5, 55.25, 58.25, 59.54999923706055, 56.150001525878906, 57.5, 61.79999923706055, 62.54999923706055, 57.45000076293945, 57.599998474121094, 54.349998474121094, 52.849998474121094, 58.5, 58.5, 62.099998474121094, 60.45000076293945, 58.650001525878906, 59.45000076293945, 58.349998474121094, 58.29999923706055, 53.20000076293945, 56.099998474121094, 55.25, 57.29999923706055, 60.54999923706055, 61.849998474121094, 62.099998474121094, 59.150001525878906, 61.150001525878906, 61.650001525878906, 57.349998474121094, 57.599998474121094, 54.849998474121094, 59.29999923706055, 60.54999923706055, 56.5, 59.04999923706055, 60.0, 60.5, 61.95000076293945, 61.75, 61.400001525878906, 63.95000076293945, 62.900001525878906, 63.5, 60.650001525878906, 60.45000076293945, 60.599998474121094, 61.29999923706055, 61.95000076293945, 60.25, 61.599998474121094, 60.75, 60.79999923706055, 61.400001525878906, 62.099998474121094, 62.0, 63.0, 61.29999923706055, 59.70000076293945, 61.400001525878906, 61.79999923706055, 60.95000076293945, 59.54999923706055, 61.25, 61.25, 58.20000076293945, 59.599998474121094, 57.150001525878906, 60.5, 59.29999923706055, 59.04999923706055, 59.95000076293945, 58.150001525878906, 59.849998474121094, 61.45000076293945, 59.599998474121094, 59.0, 60.29999923706055, 61.29999923706055, 60.54999923706055, 59.20000076293945, 59.150001525878906, 58.45000076293945, 61.04999923706055, 59.150001525878906, 62.54999923706055, 60.650001525878906, 58.75, 60.0, 60.349998474121094, 60.75, 60.25, 61.599998474121094, 64.1500015258789, 62.20000076293945, 60.75, 59.150001525878906, 58.45000076293945, 59.650001525878906, 59.900001525878906, 59.900001525878906, 58.29999923706055, 57.20000076293945, 56.349998474121094, 55.45000076293945, 57.04999923706055, 57.599998474121094, 58.599998474121094, 58.650001525878906, 54.849998474121094, 57.75, 57.5, 57.650001525878906, 57.29999923706055, 58.900001525878906, 57.0, 58.400001525878906, 58.79999923706055, 59.79999923706055, 59.95000076293945, 58.70000076293945, 60.29999923706055, 60.349998474121094, 62.20000076293945, 61.349998474121094, 59.599998474121094, 57.20000076293945, 59.75, 61.0, 60.29999923706055, 59.0, 60.25, 60.650001525878906, 61.75, 61.400001525878906, 62.79999923706055, 62.25, 62.04999923706055, 61.04999923706055, 62.79999923706055, 62.29999923706055, 61.45000076293945, 57.75, 60.04999923706055, 61.70000076293945, 59.75, 61.599998474121094, 62.20000076293945, 62.5, 63.04999923706055, 62.400001525878906, 64.30000305175781, 63.400001525878906, 64.4000015258789, 61.849998474121094, 61.45000076293945, 61.04999923706055, 62.04999923706055, 61.45000076293945, 61.25, 60.099998474121094, 59.29999923706055, 59.349998474121094, 60.20000076293945, 59.79999923706055, 54.849998474121094, 59.79999923706055, 56.849998474121094, 57.599998474121094, 57.70000076293945, 57.150001525878906, 56.75, 53.5, 57.45000076293945, 56.0, 59.0, 59.0, 58.54999923706055, 60.5, 60.900001525878906, 59.29999923706055, 55.95000076293945, 60.150001525878906, 61.54999923706055, 61.849998474121094, 59.45000076293945, 58.099998474121094, 59.099998474121094, 61.04999923706055, 58.70000076293945, 63.599998474121094, 64.19999694824219, 58.5, 63.599998474121094, 62.349998474121094, 63.5, 64.1500015258789, 64.19999694824219, 64.3499984741211, 63.849998474121094, 64.25, 58.5, 59.150001525878906, 59.70000076293945, 58.650001525878906, 58.599998474121094, 61.20000076293945, 62.900001525878906, 63.400001525878906, 62.20000076293945, 58.900001525878906, 61.849998474121094, 61.70000076293945, 62.400001525878906, 59.04999923706055, 62.29999923706055, 61.75, 63.650001525878906, 62.20000076293945, 63.599998474121094, 64.5, 63.70000076293945, 64.05000305175781, 65.05000305175781, 64.80000305175781, 64.9000015258789, 64.80000305175781, 65.0999984741211, 63.400001525878906, 64.80000305175781, 65.1500015258789, 63.400001525878906, 59.5, 63.0, 62.650001525878906, 60.70000076293945, 62.400001525878906, 63.29999923706055, 62.70000076293945, 63.04999923706055, 62.650001525878906, 57.849998474121094, 57.650001525878906, 55.04999923706055, 58.29999923706055, 57.099998474121094, 61.45000076293945, 63.150001525878906, 59.29999923706055, 60.5, 61.349998474121094, 58.650001525878906, 58.75, 58.54999923706055, 59.099998474121094, 57.25, 61.900001525878906, 62.04999923706055, 61.349998474121094, 60.95000076293945, 61.900001525878906, 63.29999923706055, 62.54999923706055, 62.29999923706055, 58.650001525878906, 60.95000076293945, 59.349998474121094, 61.150001525878906, 60.349998474121094, 60.95000076293945, 61.599998474121094, 57.349998474121094, 58.650001525878906, 59.400001525878906, 59.54999923706055, 59.75, 60.54999923706055, 62.79999923706055, 61.5, 59.400001525878906, 60.849998474121094, 60.79999923706055, 58.849998474121094, 60.70000076293945, 58.70000076293945, 58.54999923706055, 57.29999923706055, 60.150001525878906, 60.70000076293945, 60.29999923706055, 62.099998474121094, 59.0, 60.70000076293945, 61.45000076293945, 59.70000076293945, 58.349998474121094, 58.75, 61.54999923706055, 60.400001525878906, 58.70000076293945, 61.599998474121094, 59.900001525878906, 61.849998474121094, 61.75, 60.04999923706055, 58.79999923706055, 60.45000076293945, 59.849998474121094, 59.650001525878906, 61.75, 58.54999923706055, 60.400001525878906, 63.150001525878906, 65.0, 63.150001525878906, 63.29999923706055, 63.70000076293945, 65.1500015258789, 63.20000076293945, 62.95000076293945, 61.20000076293945, 63.349998474121094]
Cluster_3_accuracy_50_small=[42.30999984741211, 51.75, 6.300000190734863, 11.5, 57.099998474121094, 24.649999618530273, 64.55000305175781, 54.25, 56.0, 54.0, 60.5, 53.29999923706055, 59.099998474121094, 52.04999923706055, 54.650001525878906, 56.95000076293945, 62.099998474121094, 66.19999694824219, 52.099998474121094, 57.5, 55.849998474121094, 62.849998474121094, 57.650001525878906, 45.349998474121094, 56.25, 52.29999923706055, 57.5, 56.900001525878906, 52.45000076293945, 57.599998474121094, 65.94999694824219, 53.75, 60.099998474121094, 61.0, 59.599998474121094, 66.3499984741211, 60.099998474121094, 70.44999694824219, 67.69999694824219, 60.75, 68.44999694824219, 60.099998474121094, 63.150001525878906, 71.44999694824219, 64.94999694824219, 66.3499984741211, 70.8499984741211, 70.30000305175781, 72.8499984741211, 64.69999694824219, 70.75, 67.19999694824219, 68.30000305175781, 63.04999923706055, 64.80000305175781, 67.5, 64.19999694824219, 66.6500015258789, 69.4000015258789, 68.55000305175781, 69.6500015258789, 67.25, 63.29999923706055, 70.30000305175781, 68.69999694824219, 67.05000305175781, 73.6500015258789, 70.3499984741211, 65.80000305175781, 66.19999694824219, 65.0999984741211, 66.80000305175781, 71.44999694824219, 65.4000015258789, 69.75, 69.75, 66.9000015258789, 72.25, 68.6500015258789, 69.0, 63.5, 68.55000305175781, 69.4000015258789, 69.69999694824219, 61.400001525878906, 64.80000305175781, 69.4000015258789, 70.44999694824219, 72.3499984741211, 70.4000015258789, 72.0999984741211, 71.3499984741211, 72.5999984741211, 71.80000305175781, 73.4000015258789, 71.6500015258789, 71.5999984741211, 71.94999694824219, 73.9000015258789, 71.55000305175781, 71.0999984741211, 63.650001525878906, 67.05000305175781, 68.3499984741211, 68.55000305175781, 67.69999694824219, 71.44999694824219, 74.5, 73.0999984741211, 72.1500015258789, 70.80000305175781, 71.1500015258789, 74.6500015258789, 74.25, 73.0, 61.849998474121094, 66.5, 70.3499984741211, 69.44999694824219, 71.9000015258789, 70.30000305175781, 67.69999694824219, 68.19999694824219, 71.25, 71.6500015258789, 67.55000305175781, 69.3499984741211, 68.44999694824219, 71.0, 68.3499984741211, 71.19999694824219, 73.44999694824219, 72.69999694824219, 72.5, 74.30000305175781, 73.0999984741211, 70.25, 72.3499984741211, 70.30000305175781, 71.4000015258789, 72.9000015258789, 72.5, 72.25, 72.25, 75.30000305175781, 75.75, 74.30000305175781, 73.30000305175781, 72.4000015258789, 72.5999984741211, 71.80000305175781, 71.9000015258789, 73.80000305175781, 72.5, 74.9000015258789, 75.25, 73.8499984741211, 71.05000305175781, 73.4000015258789, 72.3499984741211, 73.0999984741211, 72.94999694824219, 72.0, 72.30000305175781, 71.94999694824219, 71.80000305175781, 74.1500015258789, 70.75, 73.4000015258789, 71.9000015258789, 72.6500015258789, 73.0999984741211, 73.25, 74.5, 73.5, 68.94999694824219, 70.4000015258789, 72.1500015258789, 69.5, 72.55000305175781, 70.19999694824219, 72.44999694824219, 70.1500015258789, 69.5, 70.30000305175781, 67.6500015258789, 70.69999694824219, 70.4000015258789, 69.5, 70.19999694824219, 68.80000305175781, 69.1500015258789, 66.9000015258789, 69.55000305175781, 69.1500015258789, 69.0, 71.5999984741211, 71.0999984741211, 69.19999694824219, 67.94999694824219, 68.75, 71.0999984741211, 70.30000305175781, 71.0, 71.69999694824219, 70.44999694824219, 73.30000305175781, 73.9000015258789, 71.8499984741211, 71.1500015258789, 71.0, 71.30000305175781, 71.6500015258789, 72.19999694824219, 71.94999694824219, 72.3499984741211, 70.5, 69.4000015258789, 73.44999694824219, 72.9000015258789, 70.25, 70.69999694824219, 70.0999984741211, 71.0999984741211, 71.0999984741211, 69.5, 69.9000015258789, 69.30000305175781, 70.9000015258789, 70.80000305175781, 72.44999694824219, 72.30000305175781, 71.5, 71.94999694824219, 70.0, 70.8499984741211, 72.1500015258789, 70.3499984741211, 72.0999984741211, 67.9000015258789, 69.75, 69.25, 70.94999694824219, 70.6500015258789, 71.4000015258789, 72.44999694824219, 72.75, 71.1500015258789, 72.1500015258789, 70.4000015258789, 70.8499984741211, 71.6500015258789, 72.0, 71.5, 71.5, 70.30000305175781, 67.1500015258789, 65.8499984741211, 67.0999984741211, 66.55000305175781, 67.5999984741211, 67.94999694824219, 70.4000015258789, 71.6500015258789, 72.80000305175781, 73.6500015258789, 72.1500015258789, 73.0, 73.25, 73.69999694824219, 73.9000015258789, 72.44999694824219, 71.4000015258789, 72.8499984741211, 72.75, 73.4000015258789, 72.3499984741211, 74.0999984741211, 73.6500015258789, 74.9000015258789, 74.19999694824219, 74.25, 73.3499984741211, 73.0999984741211, 73.0999984741211, 73.19999694824219, 72.44999694824219, 72.69999694824219, 73.5, 72.44999694824219, 73.5999984741211, 74.6500015258789, 74.1500015258789, 74.30000305175781, 73.3499984741211, 73.4000015258789, 75.3499984741211, 75.0999984741211, 73.44999694824219, 73.6500015258789, 71.9000015258789, 73.0999984741211, 71.1500015258789, 72.25, 73.0, 73.30000305175781, 72.9000015258789, 73.5999984741211, 73.25, 70.80000305175781, 71.30000305175781, 70.5, 70.0, 69.0999984741211, 69.75, 68.69999694824219, 70.55000305175781, 71.44999694824219, 69.25, 70.9000015258789, 72.80000305175781, 71.5999984741211, 74.05000305175781, 72.5, 73.6500015258789, 73.4000015258789, 71.80000305175781, 72.80000305175781, 73.6500015258789, 75.1500015258789, 73.30000305175781, 74.0, 74.30000305175781, 73.3499984741211, 75.0, 72.3499984741211, 75.55000305175781, 74.9000015258789, 74.6500015258789, 73.8499984741211, 72.8499984741211, 74.19999694824219, 73.3499984741211, 72.9000015258789, 71.75, 73.75, 72.19999694824219, 72.3499984741211, 73.55000305175781, 72.05000305175781, 72.75, 72.9000015258789, 71.30000305175781, 72.19999694824219, 72.4000015258789, 72.55000305175781, 71.55000305175781, 75.0999984741211, 75.6500015258789, 75.44999694824219, 74.44999694824219, 74.94999694824219, 74.5999984741211, 75.9000015258789, 74.30000305175781, 74.4000015258789, 74.1500015258789, 75.5999984741211, 73.8499984741211, 74.0999984741211, 75.80000305175781, 74.5999984741211, 74.3499984741211, 73.6500015258789, 74.9000015258789, 75.4000015258789, 75.30000305175781, 74.55000305175781, 75.19999694824219, 75.25, 74.4000015258789, 74.55000305175781, 74.75, 75.0, 75.1500015258789, 74.19999694824219, 75.30000305175781, 74.4000015258789, 75.25, 76.44999694824219, 75.5999984741211, 76.25, 74.8499984741211, 75.4000015258789, 74.1500015258789, 73.9000015258789, 74.9000015258789, 74.6500015258789, 73.3499984741211, 74.69999694824219, 74.4000015258789, 75.30000305175781, 76.0999984741211, 76.4000015258789, 75.05000305175781, 74.0999984741211, 73.94999694824219, 75.3499984741211, 74.75, 71.0999984741211, 74.1500015258789, 73.3499984741211, 73.94999694824219, 75.0999984741211, 76.25, 75.69999694824219, 75.4000015258789, 74.75, 76.19999694824219, 74.94999694824219, 75.19999694824219, 75.5999984741211, 74.75, 75.9000015258789, 75.05000305175781, 75.55000305175781, 74.8499984741211, 75.9000015258789, 75.44999694824219, 75.9000015258789, 75.69999694824219, 76.4000015258789, 76.55000305175781, 77.05000305175781, 76.05000305175781, 77.5, 75.9000015258789, 75.75, 76.69999694824219, 76.05000305175781, 76.9000015258789, 76.19999694824219, 74.69999694824219, 77.55000305175781, 75.6500015258789, 76.80000305175781, 75.8499984741211, 77.55000305175781, 76.69999694824219, 76.0, 76.0999984741211, 78.0999984741211, 77.75, 77.19999694824219, 77.5999984741211, 77.6500015258789, 77.69999694824219, 78.05000305175781, 76.80000305175781, 78.5, 77.75, 77.5999984741211, 76.8499984741211, 75.19999694824219, 76.0999984741211, 76.25, 75.4000015258789, 74.05000305175781, 76.69999694824219, 76.05000305175781, 76.4000015258789, 75.0, 77.1500015258789, 76.19999694824219, 75.55000305175781, 75.94999694824219, 76.19999694824219, 76.25, 76.5999984741211, 76.19999694824219, 77.3499984741211, 77.8499984741211, 76.8499984741211, 77.05000305175781, 75.1500015258789, 76.4000015258789, 74.75, 73.4000015258789, 74.4000015258789, 74.25, 75.05000305175781, 74.05000305175781, 73.80000305175781, 73.19999694824219, 73.3499984741211, 73.0999984741211, 73.30000305175781, 72.55000305175781, 73.5999984741211, 74.05000305175781]
Cluster_4_accuracy_50_small=[0.6499999761581421, 38.099998474121094, 58.04999923706055, 59.150001525878906, 64.75, 63.20000076293945, 61.099998474121094, 59.599998474121094, 67.44999694824219, 66.6500015258789, 60.54999923706055, 60.20000076293945, 65.5, 66.19999694824219, 64.19999694824219, 69.1500015258789, 61.849998474121094, 67.0, 69.5, 69.55000305175781, 68.80000305175781, 68.8499984741211, 67.0999984741211, 69.55000305175781, 68.0, 68.4000015258789, 66.3499984741211, 67.69999694824219, 64.6500015258789, 66.8499984741211, 63.400001525878906, 63.20000076293945, 64.3499984741211, 63.900001525878906, 67.30000305175781, 68.75, 65.6500015258789, 67.80000305175781, 59.349998474121094, 61.650001525878906, 64.8499984741211, 61.25, 62.5, 61.95000076293945, 63.900001525878906, 67.3499984741211, 65.05000305175781, 66.30000305175781, 64.55000305175781, 63.04999923706055, 66.75, 65.30000305175781, 59.45000076293945, 63.849998474121094, 58.75, 64.55000305175781, 64.5999984741211, 65.0, 66.0, 66.4000015258789, 65.30000305175781, 63.599998474121094, 62.45000076293945, 60.5, 62.400001525878906, 59.5, 52.650001525878906, 52.75, 57.099998474121094, 61.45000076293945, 63.79999923706055, 64.44999694824219, 66.8499984741211, 67.19999694824219, 56.29999923706055, 69.55000305175781, 61.599998474121094, 64.25, 58.5, 60.25, 57.150001525878906, 55.45000076293945, 58.349998474121094, 67.1500015258789, 63.04999923706055, 63.150001525878906, 67.1500015258789, 65.5999984741211, 65.69999694824219, 63.20000076293945, 63.25, 61.650001525878906, 62.20000076293945, 60.04999923706055, 62.20000076293945, 61.45000076293945, 62.20000076293945, 58.099998474121094, 61.349998474121094, 62.54999923706055, 63.79999923706055, 59.95000076293945, 67.25, 66.30000305175781, 62.5, 59.45000076293945, 60.70000076293945, 63.45000076293945, 59.349998474121094, 62.04999923706055, 64.55000305175781, 63.25, 59.45000076293945, 62.599998474121094, 57.650001525878906, 60.849998474121094, 64.75, 54.0, 65.44999694824219, 62.25, 69.75, 68.6500015258789, 64.8499984741211, 69.0, 67.75, 68.0999984741211, 69.1500015258789, 66.94999694824219, 69.0, 67.80000305175781, 66.5, 67.8499984741211, 66.5999984741211, 69.6500015258789, 67.19999694824219, 68.94999694824219, 65.30000305175781, 68.30000305175781, 67.8499984741211, 66.19999694824219, 66.0999984741211, 65.8499984741211, 66.94999694824219, 67.5, 65.75, 59.599998474121094, 66.44999694824219, 67.19999694824219, 66.1500015258789, 67.44999694824219, 67.9000015258789, 62.5, 61.95000076293945, 62.75, 68.19999694824219, 65.94999694824219, 65.0999984741211, 55.150001525878906, 59.70000076293945, 61.54999923706055, 60.849998474121094, 59.45000076293945, 61.849998474121094, 59.04999923706055, 57.54999923706055, 55.04999923706055, 58.54999923706055, 59.349998474121094, 59.75, 60.400001525878906, 65.30000305175781, 64.55000305175781, 60.0, 56.95000076293945, 57.650001525878906, 62.900001525878906, 66.05000305175781, 62.900001525878906, 63.400001525878906, 60.599998474121094, 58.900001525878906, 60.45000076293945, 64.5, 65.69999694824219, 58.70000076293945, 64.75, 57.79999923706055, 51.95000076293945, 60.29999923706055, 65.5999984741211, 66.0, 69.0999984741211, 70.30000305175781, 68.75, 69.30000305175781, 66.8499984741211, 68.5999984741211, 66.69999694824219, 62.04999923706055, 70.75, 70.4000015258789, 70.9000015258789, 70.19999694824219, 71.19999694824219, 72.5999984741211, 69.69999694824219, 67.5, 67.19999694824219, 68.55000305175781, 67.94999694824219, 68.44999694824219, 68.75, 69.69999694824219, 68.9000015258789, 68.0, 69.80000305175781, 67.1500015258789, 67.94999694824219, 66.44999694824219, 64.69999694824219, 58.5, 65.9000015258789, 63.20000076293945, 60.25, 61.25, 61.150001525878906, 64.25, 65.5999984741211, 67.94999694824219, 62.79999923706055, 64.69999694824219, 63.5, 61.75, 62.70000076293945, 60.95000076293945, 66.0, 65.80000305175781, 63.79999923706055, 66.05000305175781, 64.0999984741211, 63.79999923706055, 65.80000305175781, 66.0, 63.04999923706055, 64.1500015258789, 64.05000305175781, 64.05000305175781, 64.5, 61.79999923706055, 67.1500015258789, 67.25, 69.05000305175781, 65.55000305175781, 65.4000015258789, 67.05000305175781, 65.9000015258789, 66.19999694824219, 60.900001525878906, 63.29999923706055, 67.0, 67.0999984741211, 66.5, 64.0, 66.05000305175781, 64.55000305175781, 64.6500015258789, 65.69999694824219, 65.94999694824219, 60.45000076293945, 59.900001525878906, 61.099998474121094, 62.95000076293945, 63.04999923706055, 61.650001525878906, 59.650001525878906, 62.95000076293945, 62.75, 64.55000305175781, 65.5, 65.4000015258789, 63.650001525878906, 65.25, 67.5999984741211, 65.3499984741211, 60.70000076293945, 67.0, 66.05000305175781, 65.0999984741211, 67.0, 64.30000305175781, 64.9000015258789, 62.95000076293945, 60.79999923706055, 62.0, 62.599998474121094, 62.79999923706055, 62.0, 61.25, 60.79999923706055, 59.599998474121094, 57.5, 58.349998474121094, 59.70000076293945, 60.95000076293945, 62.5, 62.900001525878906, 63.25, 62.25, 61.400001525878906, 60.400001525878906, 60.400001525878906, 60.25, 61.099998474121094, 61.650001525878906, 61.45000076293945, 59.400001525878906, 62.150001525878906, 64.55000305175781, 62.150001525878906, 62.79999923706055, 64.0999984741211, 62.25, 61.20000076293945, 59.70000076293945, 60.54999923706055, 61.599998474121094, 62.599998474121094, 63.5, 62.45000076293945, 60.54999923706055, 61.29999923706055, 61.75, 61.95000076293945, 63.849998474121094, 63.79999923706055, 64.3499984741211, 63.95000076293945, 65.05000305175781, 64.44999694824219, 65.94999694824219, 65.05000305175781, 65.05000305175781, 64.44999694824219, 64.80000305175781, 64.1500015258789, 63.150001525878906, 60.20000076293945, 64.9000015258789, 65.94999694824219, 66.3499984741211, 67.44999694824219, 66.6500015258789, 65.0999984741211, 64.25, 64.9000015258789, 64.19999694824219, 65.4000015258789, 65.1500015258789, 65.30000305175781, 66.0, 65.3499984741211, 64.0999984741211, 65.05000305175781, 66.80000305175781, 65.6500015258789, 65.5999984741211, 64.0999984741211, 63.54999923706055, 63.29999923706055, 66.44999694824219, 66.69999694824219, 66.0999984741211, 64.05000305175781, 65.30000305175781, 64.6500015258789, 67.25, 65.44999694824219, 67.1500015258789, 64.3499984741211, 66.05000305175781, 66.3499984741211, 63.150001525878906, 66.55000305175781, 66.75, 66.0999984741211, 68.30000305175781, 68.5999984741211, 67.6500015258789, 67.3499984741211, 67.19999694824219, 63.400001525878906, 67.94999694824219, 67.0999984741211, 68.4000015258789, 67.94999694824219, 67.6500015258789, 68.25, 67.5, 69.69999694824219, 70.6500015258789, 69.9000015258789, 67.55000305175781, 70.19999694824219, 73.1500015258789, 71.3499984741211, 69.6500015258789, 70.9000015258789, 71.30000305175781, 71.3499984741211, 70.3499984741211, 70.0999984741211, 69.55000305175781, 69.19999694824219, 68.8499984741211, 66.69999694824219, 65.94999694824219, 63.75, 67.19999694824219, 66.9000015258789, 67.6500015258789, 63.04999923706055, 68.3499984741211, 67.75, 66.25, 68.19999694824219, 69.44999694824219, 65.5, 65.4000015258789, 69.0, 70.55000305175781, 70.05000305175781, 71.19999694824219, 72.75, 73.0, 72.30000305175781, 71.55000305175781, 72.1500015258789, 72.55000305175781, 72.05000305175781, 73.5, 71.44999694824219, 73.05000305175781, 73.44999694824219, 70.8499984741211, 69.9000015258789, 71.3499984741211, 71.3499984741211, 71.80000305175781, 72.94999694824219, 73.19999694824219, 72.9000015258789, 67.44999694824219, 71.5999984741211, 71.0999984741211, 71.6500015258789, 72.4000015258789, 68.94999694824219, 66.8499984741211, 67.1500015258789, 71.05000305175781, 70.0, 70.0999984741211, 70.8499984741211, 68.94999694824219, 67.8499984741211, 68.69999694824219, 66.05000305175781, 70.5, 70.80000305175781, 71.1500015258789, 69.80000305175781, 71.5, 70.75, 71.1500015258789, 71.0, 71.0, 70.0999984741211, 67.69999694824219, 67.05000305175781, 67.9000015258789, 67.80000305175781, 68.5999984741211, 68.69999694824219, 67.69999694824219, 66.9000015258789, 67.9000015258789, 67.75, 68.6500015258789, 66.9000015258789, 67.69999694824219, 67.75, 68.05000305175781, 68.44999694824219, 70.5, 70.9000015258789, 69.6500015258789, 67.80000305175781, 70.3499984741211, 69.5999984741211, 69.1500015258789]


In [ ]:
plt.plot(range(len(Avg_acc_cluster0)), Avg_acc_cluster0,label="smaller learning rate")
#plt.plot(range(len(Cluster_0_accuracy)), Cluster_0_accuracy,label="larger learning rate")
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 0 test accuracy')
plt.xlabel('iteration no.')
plt.legend()
plt.show()
# plt.savefig('./save/fed_{}_{}_{}_C{}_iid{}.png'.format(args.dataset, args.model, args.epochs, args.frac, args.iid))
plt.plot(range(len(Avg_acc_cluster1)), Avg_acc_cluster1,label="smaller learning rate")
#plt.plot(range(len(Cluster_1_accuracy)), Cluster_1_accuracy, label="larger learning rate")
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 1 test accuracy')
plt.xlabel('iteration no.')
plt.legend()
plt.show()

plt.plot(range(len(Avg_acc_cluster2)), Avg_acc_cluster2, label="smaller learning rate")
#plt.plot(range(len(Cluster_2_accuracy)), Cluster_2_accuracy, label="larger learning rate")
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 2 test accuracy')
plt.xlabel('iteration no.')
plt.legend()
plt.show()
plt.plot(range(len(Avg_acc_cluster3)), Avg_acc_cluster3, label="smaller learning rate" )
#plt.plot(range(len(Cluster_3_accuracy)), Cluster_3_accuracy, label="larger learning rate")
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 3 test accuracy')
plt.xlabel('iteration no.')
plt.legend()
plt.show()
plt.plot(range(len(Avg_acc_cluster4)), Avg_acc_cluster4,label="smaller learning rate")
#plt.plot(range(len(Cluster_4_accuracy)), Cluster_4_accuracy, label="larger learning rate")
#plt.ylabel('cluster 0 accuracy(%)')
plt.ylabel('cluster 4 test accuracy')
plt.xlabel('iteration no.')
plt.legend()
plt.show()

In [ ]:
print(abs_vect)
print(w_glob[2])

In [ ]:
w_glob=copy.deepcopy(w_glob_in)
print(w_glob[2])

In [ ]:
avg_acc=0
#print(sum(noise_acc[20]))
for i in range(25):
    avg_acc=avg_acc+sum(noise_acc[i])/200
#avg_acc=(sum(Avg_acc_cluster0)+sum(Avg_acc_cluster3)+sum(Avg_acc_cluster4))/(216*3)
avg_acc=avg_acc/25
print(avg_acc)

In [ ]:

print("Cluster 0 accuracy")
print(Avg_acc_cluster0)
print("Cluster 1 accuracy")
print(Avg_acc_cluster1)
print("Cluster 2 accuracy")
print(Avg_acc_cluster2)
print("Cluster 3 accuracy")
print(Avg_acc_cluster3)
print("Cluster 4 accuracy")
print(Avg_acc_cluster4)

In [ ]:
a=1e-2
print(a)